import dependencies

In [2]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
from statistics import mean, median
from pandas import read_csv
import os


Election results from OpenElections (http://openelections.net/)
Shapefile from the Utah Automated Geographic Reference Center (https://gis.utah.gov/data/political/voter-precincts/)

Several transcription errors were corrected from OpenElections using their raw results gathered from counties (https://github.com/openelections/openelections-sources-ut/tree/master/2018%20General%20Election). OE results in Rich and Morgan Counties were based on non-final reports, which were collected from county sources.

Emery County - 8 "canvas votes" were distributed to precincts.

Weber County - 6 precincts had their results suppressed, since all cast fewer than 15 votes, for a total of 42 votes. These were distributed based on the number of votes cast and the reported vote in 2016, which was not suppressed.

G18USSRROM - Mitt Romney (Republican Party)
G18USSDWIL - Jenny Wilson (Democratic Party)
G18USSIMCC - Reed C. McCandless (Independent American Party)
G18USSLBOW - Craig R. Bowden (Libertarian Party)
G18USSCAAL - Tim Aalders (Conservative Party)
G18USSOWRI - Write-in Votes

import files

In [3]:
ut_election_results = pd.read_csv('raw-from-source/20181106__ut__general__precinct.csv')
ut_vest = gp.read_file("raw-from-source/ut_2018/ut_2018.shp")
ut_geo = gp.read_file("raw-from-source/Utah_Vista_Ballot_Areas/VistaBallotAreas.shp")
ut_vest_2016 = gp.read_file("raw-from-source/ut_2016/ut_2016.shp")
ut_vest_county = pd.read_csv('raw-from-source/20181106__ut__general__county.csv')

In [29]:
ut_vest_county = ut_vest_county[ut_vest_county['office'] == 'U.S. Senate']
display(ut_vest_county['candidate'].unique())
ut_vest_county['candidate'] = ut_vest_county['candidate'].map(senator_dict)
display(ut_vest_county)

array(['Craig R. Bowden', 'Jenny Wilson', 'Mitt Romney',
       'Reed C. McCandless', 'Tim Aalders'], dtype=object)

,county,office,district,party,candidate,votes
0,Beaver,U.S. Senate,NaN,LIB,G18USSLBOW,45
1,Box Elder,U.S. Senate,NaN,LIB,G18USSLBOW,533
2,Cache,U.S. Senate,NaN,LIB,G18USSLBOW,1106
3,Carbon,U.S. Senate,NaN,LIB,G18USSLBOW,168
4,Daggett,U.S. Senate,NaN,LIB,G18USSLBOW,7
...,...,...,...,...,...,...
140,Utah,U.S. Senate,NaN,CON,G18USSCAAL,5097
141,Wasatch,U.S. Senate,NaN,CON,G18USSCAAL,272
142,Washington,U.S. Senate,NaN,CON,G18USSCAAL,3352
143,Wayne,U.S. Senate,NaN,CON,G18USSCAAL,48


In [4]:
display(ut_geo[ut_geo['PrecinctID'] == 'WLRD'])

,OBJECTID,CountyID,VistaID,PrecinctID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area,geometry
1613,1614,2,WLRD,WLRD,None,None,2013-05-31,None,myoung,2013-05-31,{A87F9F36-717D-4A67-BB7F-83D5FB0AD327},1.354522,0.047449,"POLYGON ((-112.34179 41.48166, -112.34051 41.4..."
1937,1938,2,WLRD:M,WLRD,M,None,2013-05-31,None,myoung,2013-05-31,{E21B2A5E-A1FD-4966-B6D8-D0BD7C8013FD},0.196235,0.001981,"POLYGON ((-112.04458 41.44419, -112.04454 41.4..."


In [5]:
display(ut_vest[ut_vest['CountyID'] == 1])

,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
0,1,BV01,1-BV01,402,46,8,9,19,1,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,1-BV02,310,46,3,10,17,0,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,1-BV03,244,30,0,2,12,2,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,1-BV04,84,7,1,4,23,0,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,1-GR01,56,9,6,1,1,1,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
5,1,ML01,1-ML01,38,23,0,6,0,0,"POLYGON ((324513.800 4253004.500, 324578.800 4..."
6,1,ML02,1-ML02,182,102,3,7,18,3,"POLYGON ((322814.500 4251988.700, 322875.200 4..."
7,1,ML03,1-ML03,61,16,1,3,8,1,"POLYGON ((344948.320 4270854.760, 344950.160 4..."
8,1,MV01,1-MV01,243,19,8,3,40,1,"POLYGON ((328777.760 4237537.600, 329104.140 4..."


In [6]:
display(ut_election_results)
rom = (ut_election_results[ut_election_results['candidate']=='MITT ROMNEY'])
display(rom[rom['votes'] == '603'])
print(ut_election_results.loc[ut_election_results['precinct'].str.contains("Salt Lake", case=False)])

,county,precinct,office,district,candidate,party,votes
0,Wayne,Fremont,U.S. Senate,NaN,Tim Aalders,Con,7
1,Wayne,Loa in/out,U.S. Senate,NaN,Tim Aalders,Con,6
2,Wayne,Lyman in/out,U.S. Senate,NaN,Tim Aalders,Con,6
3,Wayne,Bicknell in/out,U.S. Senate,NaN,Tim Aalders,Con,8
4,Wayne,Teesdale,U.S. Senate,NaN,Tim Aalders,Con,3
...,...,...,...,...,...,...,...
41993,Morgan,Croydon 15.5,U S House,NaN,Rob Bishop,REP,64
41994,Morgan,Croydon 15.5,U S House,NaN,Eric Eliason,UUP,8
41995,Morgan,Croydon 15.5,U S House,NaN,Adam Davis,GRN,2
41996,Morgan,Croydon 15.5,Utah State House,53.0,Logan Wilde,REP,80


,county,precinct,office,district,candidate,party,votes


Empty DataFrame
Columns: [county, precinct, office, district, candidate, party, votes]
Index: []


In [7]:
ut_senators = ut_election_results[(ut_election_results['office'] == 'U.S. Senate') | (ut_election_results['office'] == 'U S Senate')]
county_names = ut_senators['county'].unique()
print(county_names)
county_name_dict = {elem : pd.DataFrame for elem in county_names}
for key in county_name_dict.keys():
    county_name_dict[key] = ut_senators[:][ut_senators.county == key]
print(len(county_name_dict))

county_nums = list(range(1, 30))
print(county_nums)

county_name_dict_vest = {elem : pd.DataFrame for elem in county_nums}
for key in county_name_dict_vest.keys():
    county_name_dict_vest[key] = ut_vest[:][ut_vest.CountyID == key]
print(len(county_name_dict_vest))

['Wayne' 'Salt Lake' 'Wasatch' 'Iron' 'Beaver' 'Utah' 'Carbon' 'Juab'
 'Kane' 'Garfield' 'Daggett' 'Sanpete' 'Sevier' 'Box Elder' 'Grand'
 'Millard' 'San Juan' 'Tooele' 'Duchesne' 'Cache' 'Weber' 'Davis' 'Summit'
 'Uintah' 'Emery' 'Piute' 'Washington' 'Rich' 'Morgan']
29
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
29


In [8]:
senators = (ut_senators['candidate'].unique())

In [9]:
pd.options.mode.chained_assignment = None  # default='warn'


def find_votes(name):
    df = (ut_senators[ut_senators['candidate']== name])
    df.votes = pd.to_numeric(df.votes, errors='coerce')
    return (df[df['votes'] > 0])
    

for senator in senators:
    df = find_votes(senator)
    print(senator, 'has a total of', df.votes.sum(), 'in', len(df), 'precincts')

Tim Aalders has a total of 22190.0 in 1263 precincts
Craig Bowden has a total of 13082.0 in 888 precincts
Reed McCandless has a total of 5635.0 in 840 precincts
Jenny Wilson has a total of 142864.0 in 1331 precincts
Mitt Romney has a total of 456966.0 in 1346 precincts
write-in has a total of 721.0 in 293 precincts
MITT ROMNEY has a total of 226341 in 860 precincts
JENNY WILSON has a total of 189902 in 858 precincts
CRAIG R.  BOWDEN has a total of 9854 in 747 precincts
TIM AALDERS has a total of 7941 in 798 precincts
REED C. MCCANDLESS has a total of 4265 in 707 precincts
WRITE-IN has a total of 746 in 420 precincts
CALEB DAN REEVE has a total of 27 in 26 precincts
TYRONE JENSEN has a total of 5 in 4 precincts
GLADE G FITZGERALD has a total of 2 in 2 precincts
HEKTOR REIKSTHEGN has a total of 2 in 2 precincts
CODY JUDY has a total of 2 in 1 precincts
RYAN DANIEL JACKSON has a total of 1 in 1 precincts
ABE KORB has a total of 0 in 0 precincts
Ryan Jackson has a total of 1 in 1 precincts

In [10]:
senator_dict = {}
for senator in senators:
    senator_dict[senator] = None
print(senator_dict)

{'Tim Aalders': None, 'Craig Bowden': None, 'Reed McCandless': None, 'Jenny Wilson': None, 'Mitt Romney': None, 'write-in': None, 'MITT ROMNEY': None, 'JENNY WILSON': None, 'CRAIG R.  BOWDEN': None, 'TIM AALDERS': None, 'REED C. MCCANDLESS': None, 'WRITE-IN': None, 'CALEB DAN REEVE': None, 'TYRONE JENSEN': None, 'GLADE G FITZGERALD': None, 'HEKTOR REIKSTHEGN': None, 'CODY JUDY': None, 'RYAN DANIEL JACKSON': None, 'ABE KORB': None, 'Ryan Jackson': None, 'Craig R. Bowden': None, 'Reed C. McCandless': None, 'Glade G Fitzgerald (W)': None, 'Caleb Dan Reeve (W)': None, 'Hektor Reiksthegn (W)': None, 'Abe Korb (W)': None, 'Ryan Daniel Jackson (W)': None, 'Cody Judy (W)': None, 'Tyrone Jensen (W)': None, 'Write In': None, 'Write-In Totals': None, 'write-ins': None, 'Write-In Total': None, 'CRAIG R. BOWDEN': None, 'REED C. McCANDLESS': None, 'Write-ins': None, 'Write-In': None, 'CRAIG R BOWDEN': None, 'REED C MCCANDLESS': None, 'Caleb Dan Reeve': None, 'Write-In Votes': None}


In [11]:
# VEST options are: 	G18USSRROM	G18USSDWIL	G18USSIMCC	G18USSLBOW	G18USSCAAL	G18USSOWRI	
# all not in major 5 parties - write-ins
senator_dict = {
    'Tim Aalders': 'G18USSCAAL',
    'Craig Bowden': 'G18USSLBOW',
    'Reed McCandless': 'G18USSIMCC',
    'Jenny Wilson': 'G18USSDWIL',
    'Mitt Romney': 'G18USSRROM',
    'write-in': 'G18USSOWRI',
    'MITT ROMNEY': 'G18USSRROM', 
    'JENNY WILSON': 'G18USSDWIL', 
    'CRAIG R.  BOWDEN': 'G18USSLBOW', 
    'TIM AALDERS': 'G18USSCAAL', 
    'REED C. MCCANDLESS': 'G18USSIMCC', 
    'WRITE-IN': 'G18USSOWRI', 
    'CALEB DAN REEVE': 'G18USSOWRI', 
    'TYRONE JENSEN': 'G18USSOWRI', 
    'GLADE G FITZGERALD': 'G18USSOWRI', 
    'HEKTOR REIKSTHEGN': 'G18USSOWRI', 
    'CODY JUDY': 'G18USSOWRI', 
    'RYAN DANIEL JACKSON': 'G18USSOWRI', 
    'ABE KORB': 'G18USSOWRI', 
    'Ryan Jackson': 'G18USSOWRI', 
    'Craig R. Bowden': 'G18USSLBOW', 
    'Reed C. McCandless': 'G18USSIMCC', 
    'Glade G Fitzgerald (W)': 'G18USSOWRI', 
    'Caleb Dan Reeve (W)': 'G18USSOWRI', 
    'Hektor Reiksthegn (W)': 'G18USSOWRI', 
    'Abe Korb (W)': 'G18USSOWRI', 
    'Ryan Daniel Jackson (W)': 'G18USSOWRI', 
    'Cody Judy (W)': 'G18USSOWRI', 
    'Tyrone Jensen (W)': 'G18USSOWRI', 
    'Write In': 'G18USSOWRI', 
    'Write-In Totals': 'G18USSOWRI', 
    'write-ins': 'G18USSOWRI', 
    'Write-In Total': 'G18USSOWRI', 
    'CRAIG R. BOWDEN': 'G18USSLBOW', 
    'REED C. McCANDLESS': 'G18USSIMCC', 
    'Write-ins': 'G18USSOWRI', 
    'Write-In': 'G18USSOWRI', 
    'CRAIG R BOWDEN': 'G18USSLBOW', 
    'REED C MCCANDLESS': 'G18USSIMCC', 
    'Caleb Dan Reeve': 'G18USSOWRI', 
    'Write-In Votes': 'G18USSOWRI'
}


In [12]:
display(ut_vest_2016)

,CountyID,PrecinctID,county,precinct,G16PRERTRU,G16PREDCLI,G16PRELJOH,G16PREISTE,G16PREIMCM,G16PREOOTH,...,G16ATGDHAR,G16ATGLMCC,G16ATGAISB,G16AUDRDOU,G16AUDDMIT,G16AUDAGRE,G16TRERDAM,G16TREDHAN,G16TRECPRO,geometry
0,1,BV01,Beaver,Beaver 1,469,46,6,1,83,3,...,48,18,17,481,69,27,454,80,36,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,Beaver,Beaver 2,334,38,4,4,74,4,...,48,25,14,345,72,21,329,90,19,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,Beaver,Beaver 3,268,22,3,0,49,2,...,31,6,11,269,44,14,254,52,18,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,Beaver,Beaver 4,101,10,3,0,23,0,...,16,9,3,92,23,12,94,19,14,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,Beaver,Greenville,76,10,0,2,4,0,...,12,0,0,67,16,0,57,20,5,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2291,7,RO81,Duchesne,RO81,216,9,2,1,18,0,...,10,15,3,216,16,8,198,14,28,"POLYGON ((580836.220 4464060.720, 580837.220 4..."
2292,7,RO82,Duchesne,RO82,75,15,2,0,13,1,...,16,6,0,72,24,4,70,21,10,"POLYGON ((586967.910 4461063.280, 586317.330 4..."
2293,7,RO91,Duchesne,RO91,0,0,0,0,2,1,...,0,0,0,3,0,0,2,0,1,"POLYGON ((583252.560 4462118.640, 583274.150 4..."
2294,7,TA11,Duchesne,TA11,197,12,3,0,17,0,...,17,4,6,175,30,11,173,26,17,"POLYGON ((533169.000 4511919.160, 533169.000 4..."


In [13]:
counties = ut_vest_2016['county'].unique()
county_name_to_num = {}
for county in counties:
    if county not in county_name_to_num:
        county_name_to_num[county] = ut_vest_2016[ut_vest_2016['county'] == county].iloc[0]['CountyID']

    
county_num_to_name = {v: k for k, v in county_name_to_num.items()}
print(county_name_to_num)
print(len(county_num_to_name))
print(county_num_to_name)
print(len(county_num_to_name))

{'Beaver': 1, 'Box Elder': 2, 'Cache': 3, 'Carbon': 4, 'Daggett': 5, 'Davis': 6, 'Emery': 8, 'Garfield': 9, 'Iron': 11, 'Kane': 13, 'Millard': 14, 'Morgan': 15, 'Piute': 16, 'Rich': 17, 'San Juan': 19, 'Sanpete': 20, 'Sevier': 21, 'Summit': 22, 'Uintah': 24, 'Wayne': 28, 'Grand': 10, 'Juab': 12, 'Salt Lake': 18, 'Tooele': 23, 'Utah': 25, 'Wasatch': 26, 'Washington': 27, 'Weber': 29, 'Duchesne': 7}
29
{1: 'Beaver', 2: 'Box Elder', 3: 'Cache', 4: 'Carbon', 5: 'Daggett', 6: 'Davis', 8: 'Emery', 9: 'Garfield', 11: 'Iron', 13: 'Kane', 14: 'Millard', 15: 'Morgan', 16: 'Piute', 17: 'Rich', 19: 'San Juan', 20: 'Sanpete', 21: 'Sevier', 22: 'Summit', 24: 'Uintah', 28: 'Wayne', 10: 'Grand', 12: 'Juab', 18: 'Salt Lake', 23: 'Tooele', 25: 'Utah', 26: 'Wasatch', 27: 'Washington', 29: 'Weber', 7: 'Duchesne'}
29


In [14]:
display(ut_senators[ut_senators['county'] == 'Weber'])

,county,precinct,office,district,candidate,party,votes
27760,Weber,CYN001,U.S. Senate,NaN,Jenny Wilson,Dem,53
27761,Weber,FRW001,U.S. Senate,NaN,Jenny Wilson,Dem,79
27762,Weber,FRW002,U.S. Senate,NaN,Jenny Wilson,Dem,132
27763,Weber,FRW003,U.S. Senate,NaN,Jenny Wilson,Dem,129
27764,Weber,FRW004,U.S. Senate,NaN,Jenny Wilson,Dem,143
...,...,...,...,...,...,...,...
28703,Weber,WHV007,U.S. Senate,NaN,write-in,NaN,3
28704,Weber,WHV008,U.S. Senate,NaN,write-in,NaN,1
28705,Weber,WNO001,U.S. Senate,NaN,write-in,NaN,*
28706,Weber,WNO002,U.S. Senate,NaN,write-in,NaN,*


In [15]:
def pivots(curr_df, county_num):
    pivoted = curr_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first').reset_index()
    pivoted['uniqueid'] = (county_num.astype(str) + '-' + pivoted['mergeid'].astype(str))
    pivoted['CountyID'] = county_num
    return pivoted

def pivot_county(county_name):
    county_df = county_name_dict[county_name] #curr county dataframe ut_senators
    county_num = county_dict[county_name] 
    print(county_num)
    curr_county_2016 = ut_vest_2016[ut_vest_2016['CountyID'] == county_num]
    curr_dict = curr_county_2016.set_index('precinct').to_dict()['PrecinctID'] #dict based of ut vest 2016 precincts
    curr_df = county_df
    curr_df['mergeid'] = curr_df['precinct'].map(curr_dict)
    non_valid_df = (curr_df[curr_df['mergeid'].isnull()]) #df of non dict mappable precincts from ut_senators
    non_pivot_list = []
    if not non_valid_df.empty:
        print(curr_dict.keys(), curr_df['precinct'].unique())
        non_pivot_list = np.setdiff1d(list(curr_dict.keys()),list(curr_df['precinct'].unique())) #list of precincts from ut vest 2016 with no matches
    curr_df['candidateid'] = curr_df['candidate'].map(senator_dict)
    return (county_df, county_num, curr_dict, non_valid_df, non_pivot_list)

In [16]:
concat_dict = {}

In [17]:
# county 1
# set up
county_num = 1
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# create dict using vest 2016
county_vest_2016 = ut_vest_2016[ut_vest_2016['CountyID'] == county_num]
county_name_to_id_dict = county_vest_2016.set_index('precinct').to_dict()['PrecinctID'] #dict based of ut vest 2016 precincts

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].map(county_name_to_id_dict)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
display(county_name_dict_vest[county_num])

#add missing rows (if any)
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches

vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)

print('\n\nprecincts that exist in ut_senators but not in vest 2018 are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

template_df = missing_row # to be used for counties without missing rows but with mods/splits, to get structure right

county num is: 1 and county name is: Beaver


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,BV01,19,46,8,9,1,402,1-BV01,1
1,BV02,17,46,3,10,0,310,1-BV02,1
2,BV03,12,30,0,2,2,244,1-BV03,1
3,BV04,23,7,1,4,0,84,1-BV04,1
4,GR01,1,9,6,1,1,56,1-GR01,1
5,ML01,0,23,0,6,0,38,1-ML01,1
6,ML02,18,102,3,7,3,182,1-ML02,1
7,ML03,8,16,1,3,1,61,1-ML03,1
8,MV01,40,19,8,3,1,243,1-MV01,1


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
0,1,BV01,1-BV01,402,46,8,9,19,1,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,1-BV02,310,46,3,10,17,0,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,1-BV03,244,30,0,2,12,2,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,1-BV04,84,7,1,4,23,0,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,1-GR01,56,9,6,1,1,1,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
5,1,ML01,1-ML01,38,23,0,6,0,0,"POLYGON ((324513.800 4253004.500, 324578.800 4..."
6,1,ML02,1-ML02,182,102,3,7,18,3,"POLYGON ((322814.500 4251988.700, 322875.200 4..."
7,1,ML03,1-ML03,61,16,1,3,8,1,"POLYGON ((344948.320 4270854.760, 344950.160 4..."
8,1,MV01,1-MV01,243,19,8,3,40,1,"POLYGON ((328777.760 4237537.600, 329104.140 4..."


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,1,BV01,1-BV01,402,46,8,9,19,1
1,1,BV02,1-BV02,310,46,3,10,17,0
2,1,BV03,1-BV03,244,30,0,2,12,2
3,1,BV04,1-BV04,84,7,1,4,23,0
4,1,GR01,1-GR01,56,9,6,1,1,1
5,1,ML01,1-ML01,38,23,0,6,0,0
6,1,ML02,1-ML02,182,102,3,7,18,3
7,1,ML03,1-ML03,61,16,1,3,8,1
8,1,MV01,1-MV01,243,19,8,3,40,1




precincts that exist in ut_senators but not in vest 2018 are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [18]:
# county 2 - Williard missing
# set up
county_num = 2
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

display(county_df[county_df['precinct'] == 'South Willard'])

# create dict using vest 2016
county_vest_2016 = ut_vest_2016[ut_vest_2016['CountyID'] == county_num]
county_name_to_id_dict = county_vest_2016.set_index('precinct').to_dict()['PrecinctID'] #dict based of ut vest 2016 precincts

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].map(county_name_to_id_dict)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# additional step of separating precincts who did not match directly with vest 2016 precinct name -> id 
non_valid_df = (county_df[county_df['mergeid'].isnull()]) #df of non dict mappable precincts from ut_senators
display(non_valid_df)
non_valid_array = np.setdiff1d(list(county_name_to_id_dict.keys()),list(county_df['precinct'].unique())) #list of precincts from ut vest 2016 with no matches
vest_array = non_valid_df['precinct'].unique()
print('\n\nprecincts that exist in ut_senators but not in 2016 dict are:', vest_array)
print('\n\nprecincts that exist in 2016 dict but not in ut_senators are:', non_valid_array)
# by observing the difference in two lists above, deduce the changes required in the dict to properly translate

non_valid_list = list(non_valid_array)
vest_list = list(vest_array)

# Tremonton 04 has a transcription issue in the VEST 2016 data
non_valid_list.remove('Trementon 04')
vest_list.remove('Tremonton 4') 

#zip two files in alphabetical order
trans_dict = dict(zip(non_valid_list, vest_list))
trans_dict['Trementon 04'] = 'Tremonton 4'

display(county_vest_2016[county_vest_2016['precinct'] == 'Willard'])

new_dict = county_name_to_id_dict

for key, value in trans_dict.items():
    new_dict[value] = new_dict.pop(key)
    
county_df['mergeid'] = county_df['precinct'].map(new_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]['uniqueid']))
display(county_name_dict_vest[county_num])

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['mergeid'] == 'WLRD'] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)

concat_dict[county_num] = final_df

county num is: 2 and county name is: Box Elder


,county,precinct,office,district,candidate,party,votes
24249,Box Elder,South Willard,U.S. Senate,NaN,TIM AALDERS,CON,25
24250,Box Elder,South Willard,U.S. Senate,NaN,CRAIG R. BOWDEN,LIB,20
24251,Box Elder,South Willard,U.S. Senate,NaN,REED C. McCANDLESS,IAP,6
24252,Box Elder,South Willard,U.S. Senate,NaN,JENNY WILSON,DEM,81
24253,Box Elder,South Willard,U.S. Senate,NaN,MITT ROMNEY,REP,463
24254,Box Elder,South Willard,U.S. Senate,NaN,Write-ins,NaN,2


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
23968,Box Elder,Fielding 1,U.S. Senate,NaN,TIM AALDERS,CON,9,NaN,G18USSCAAL
23969,Box Elder,Fielding 1,U.S. Senate,NaN,CRAIG R. BOWDEN,LIB,2,NaN,G18USSLBOW
23970,Box Elder,Fielding 1,U.S. Senate,NaN,REED C. McCANDLESS,IAP,0,NaN,G18USSIMCC
23971,Box Elder,Fielding 1,U.S. Senate,NaN,JENNY WILSON,DEM,14,NaN,G18USSDWIL
23972,Box Elder,Fielding 1,U.S. Senate,NaN,MITT ROMNEY,REP,138,NaN,G18USSRROM
...,...,...,...,...,...,...,...,...,...
24324,Box Elder,Tremonton 5,U.S. Senate,NaN,CRAIG R. BOWDEN,LIB,10,NaN,G18USSLBOW
24325,Box Elder,Tremonton 5,U.S. Senate,NaN,REED C. McCANDLESS,IAP,5,NaN,G18USSIMCC
24326,Box Elder,Tremonton 5,U.S. Senate,NaN,JENNY WILSON,DEM,44,NaN,G18USSDWIL
24327,Box Elder,Tremonton 5,U.S. Senate,NaN,MITT ROMNEY,REP,290,NaN,G18USSRROM




precincts that exist in ut_senators but not in 2016 dict are: ['Fielding 1' 'Fielding 2' 'Garland 1' 'Garland 2' 'Perry 1' 'Perry 2'
 'Perry 3' 'Perry 4' 'Tremonton 1' 'Tremonton 2' 'Tremonton 3'
 'Tremonton 4' 'Tremonton 5']


precincts that exist in 2016 dict but not in ut_senators are: ['Fielding 01' 'Fielding 02' 'Garland 01' 'Garland 02' 'Perry 01'
 'Perry 02' 'Perry 03' 'Perry 04' 'Trementon 04' 'Tremonton 01'
 'Tremonton 02' 'Tremonton 03' 'Tremonton 05' 'Willard']


,CountyID,PrecinctID,county,precinct,G16PRERTRU,G16PREDCLI,G16PRELJOH,G16PREISTE,G16PREIMCM,G16PREOOTH,...,G16ATGDHAR,G16ATGLMCC,G16ATGAISB,G16AUDRDOU,G16AUDDMIT,G16AUDAGRE,G16TRERDAM,G16TREDHAN,G16TRECPRO,geometry
60,2,WLRD,Box Elder,Willard,485,96,23,3,153,12,...,110,43,36,539,142,61,518,173,55,"POLYGON ((387979.000 4593096.400, 388079.300 4..."


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,BC01,20,113,16,20,2,432,2-BC01,2
1,BC02,14,61,4,8,1,380,2-BC02,2
2,BC03,26,104,7,23,5,306,2-BC03,2
3,BC04,18,85,8,21,3,296,2-BC04,2
4,BC05,14,92,8,14,3,215,2-BC05,2
5,BC06,24,61,11,16,5,428,2-BC06,2
6,BC07,12,83,11,17,2,344,2-BC07,2
7,BC08,7,58,6,9,1,175,2-BC08,2
8,BC09,4,38,5,6,0,228,2-BC09,2
9,BC10,7,67,5,8,0,361,2-BC10,2


52 53


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
9,2,BC01,2-BC01,432,113,16,20,20,2,"POLYGON ((414524.370 4593984.900, 414522.580 4..."
10,2,BC02,2-BC02,380,61,4,8,14,1,"POLYGON ((414555.830 4595123.510, 414524.370 4..."
11,2,BC03,2-BC03,306,104,7,23,26,5,"POLYGON ((415201.410 4594509.040, 415185.980 4..."
12,2,BC04,2-BC04,296,85,8,21,18,3,"POLYGON ((415237.300 4595917.200, 415225.500 4..."
13,2,BC05,2-BC05,215,92,8,14,14,3,"MULTIPOLYGON (((415245.680 4596194.430, 415237..."
14,2,BC06,2-BC06,428,61,11,16,24,5,"POLYGON ((411603.260 4601803.670, 411600.700 4..."
15,2,BC07,2-BC07,344,83,11,17,12,2,"POLYGON ((415311.780 4598727.060, 415296.070 4..."
16,2,BC08,2-BC08,175,58,6,9,7,1,"POLYGON ((415664.790 4597477.770, 415634.900 4..."
17,2,BC09,2-BC09,228,38,5,6,4,0,"POLYGON ((416035.600 4596274.800, 416036.550 4..."
18,2,BC10,2-BC10,361,67,5,8,7,0,"POLYGON ((416713.460 4597882.240, 419201.140 4..."


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,2,WLRD,2-WLRD,609,126,13,11,26,5
1,2,BC01,2-BC01,432,113,16,20,20,2
2,2,BC02,2-BC02,380,61,4,8,14,1
3,2,BC03,2-BC03,306,104,7,23,26,5
4,2,BC04,2-BC04,296,85,8,21,18,3
5,2,BC05,2-BC05,215,92,8,14,14,3
6,2,BC06,2-BC06,428,61,11,16,24,5
7,2,BC07,2-BC07,344,83,11,17,12,2
8,2,BC08,2-BC08,175,58,6,9,7,1
9,2,BC09,2-BC09,228,38,5,6,4,0


In [19]:
# county 3

def split_id(string):
    split = string.split('-', 1)
    if len(split) > 1:
        output = split[0] + ':' + split[1]
        return output
    return string

# set up
county_num = 3
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators

print('county num is:', county_num, 'and county name is:', county_name)
# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(split_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[(vest18['mergeid'] == 'UNINH') | (vest18['mergeid'] == 'UNINHC')] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([cleaned_df, missing_row]).reset_index().drop(columns='index')

display(final_df)

#rename transcription errors

unique_id_dict = {
    '3-LOG01:1': '3-LOG01:I',
    '3-LOG27:1': '3-LOG27:I',
    '3-LOG29:1': '3-LOG29:I',
    '3-LOG33:1': '3-LOG33:I',
    '3-LOG37:CSD': '3-LOG27:CSD'
}

merge_id_dict = {
    'LOG01:1': 'LOG01:I',
    'LOG27:1': 'LOG27:I',
    'LOG29:1': 'LOG29:I',
    'LOG33:1': 'LOG33:I',
    'LOG37:CSD': 'LOG27:CSD'
}

final_df['uniqueid'] = final_df['uniqueid'].replace(unique_id_dict)
final_df['mergeid'] = final_df['mergeid'].replace(merge_id_dict)

display(final_df)

missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)


concat_dict[county_num] = final_df

county num is: 3 and county name is: Cache


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AMA,11,19,3,1,0,151,3-AMA,3
1,BEN,6,35,2,3,0,224,3-BEN,3
2,CLK,11,27,1,1,6,218,3-CLK,3
3,COR,5,9,0,2,4,87,3-COR,3
4,COV,11,24,5,8,2,180,3-COV,3
...,...,...,...,...,...,...,...,...,...
87,TRE,11,23,2,6,0,150,3-TRE,3
88,WEL01,16,28,3,7,2,311,3-WEL01,3
89,WEL02,33,56,1,7,1,417,3-WEL02,3
90,WEL03,13,34,2,6,2,319,3-WEL03,3


92 94 92


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
62,3,AMA,3-AMA,151,19,3,1,11,0,"POLYGON ((426961.500 4639081.600, 426914.290 4..."
63,3,BEN,3-BEN,224,35,2,3,6,0,"POLYGON ((428268.470 4632239.700, 428258.320 4..."
64,3,CLK,3-CLK,218,27,1,1,11,6,"MULTIPOLYGON (((420537.600 4643454.280, 420532..."
65,3,COR,3-COR,87,9,0,2,5,4,"POLYGON ((419299.400 4650076.310, 419301.580 4..."
66,3,COV,3-COV,180,24,5,8,11,2,"POLYGON ((438724.850 4649949.880, 438727.340 4..."
...,...,...,...,...,...,...,...,...,...,...
151,3,WEL02,3-WEL02,417,56,1,7,33,1,"POLYGON ((421354.840 4609998.250, 421684.850 4..."
152,3,WEL03,3-WEL03,319,34,2,6,13,2,"POLYGON ((422407.680 4614481.010, 422420.640 4..."
153,3,WEL04,3-WEL04,445,66,3,13,14,0,"POLYGON ((424766.080 4610401.270, 424765.380 4..."
2261,3,UNINH,3-UNINH,0,0,0,0,0,0,"MULTIPOLYGON (((435554.350 4631146.260, 435550..."




precincts that exist in ut_senators but not in vest 2018  are: ['3-LOG01:1' '3-LOG27:1' '3-LOG29:1' '3-LOG33:1' '3-LOG37:CSD']


precincts that exist in vest 2018 but not in ut senators are: ['3-LOG01:I' '3-LOG27:CSD' '3-LOG27:I' '3-LOG29:I' '3-LOG33:I' '3-UNINH'
 '3-UNINHC']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
2261,3,UNINH,3-UNINH,0,0,0,0,0,0
2262,3,UNINHC,3-UNINHC,0,0,0,0,0,0


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AMA,11,19,3,1,0,151,3-AMA,3
1,BEN,6,35,2,3,0,224,3-BEN,3
2,CLK,11,27,1,1,6,218,3-CLK,3
3,COR,5,9,0,2,4,87,3-COR,3
4,COV,11,24,5,8,2,180,3-COV,3
...,...,...,...,...,...,...,...,...,...
89,WEL02,33,56,1,7,1,417,3-WEL02,3
90,WEL03,13,34,2,6,2,319,3-WEL03,3
91,WEL04,14,66,3,13,0,445,3-WEL04,3
92,UNINH,0,0,0,0,0,0,3-UNINH,3


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AMA,11,19,3,1,0,151,3-AMA,3
1,BEN,6,35,2,3,0,224,3-BEN,3
2,CLK,11,27,1,1,6,218,3-CLK,3
3,COR,5,9,0,2,4,87,3-COR,3
4,COV,11,24,5,8,2,180,3-COV,3
...,...,...,...,...,...,...,...,...,...
89,WEL02,33,56,1,7,1,417,3-WEL02,3
90,WEL03,13,34,2,6,2,319,3-WEL03,3
91,WEL04,14,66,3,13,0,445,3-WEL04,3
92,UNINH,0,0,0,0,0,0,3-UNINH,3




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [20]:
# county 4

pd.set_option('display.max_rows', 10)

def strip_id(string):
    output = string[0:2]
    return str(int(output))

# set up
county_num = 4
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# create dict using vest 2016

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

# by observing the difference in two lists above, deduce the changes required in the dict to properly translate
#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 4 and county name is: Carbon


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,16,16,138,11,11,1,253,4-16,4
1,17,8,102,16,16,1,214,4-17,4
2,19,15,95,20,13,2,326,4-19,4
3,2,0,7,4,1,0,14,4-2,4
4,20,22,195,23,9,2,255,4-20,4
...,...,...,...,...,...,...,...,...,...
15,37,26,160,20,19,1,327,4-37,4
16,38,13,115,12,4,1,305,4-38,4
17,39,13,96,6,7,1,353,4-39,4
18,40,0,37,6,6,0,83,4-40,4


20 20 20


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
154,4,16,4-16,253,138,11,11,16,1,"POLYGON ((515397.780 4383729.690, 515419.780 4..."
155,4,17,4-17,214,102,16,16,8,1,"POLYGON ((518639.910 4382677.630, 518645.140 4..."
156,4,19,4-19,326,95,20,13,15,2,"POLYGON ((521528.550 4377617.190, 522335.170 4..."
157,4,2,4-2,14,7,4,1,0,0,"POLYGON ((485989.850 4397309.520, 485992.950 4..."
158,4,20,4-20,255,195,23,9,22,2,"POLYGON ((550649.710 4378300.640, 551057.640 4..."
...,...,...,...,...,...,...,...,...,...,...
169,4,37,4-37,327,160,20,19,26,1,"POLYGON ((523777.460 4406970.830, 523776.320 4..."
170,4,38,4-38,305,115,12,4,13,1,"POLYGON ((515743.650 4388345.430, 515941.660 4..."
171,4,39,4-39,353,96,6,7,13,1,"POLYGON ((512024.490 4388326.760, 512055.710 4..."
172,4,40,4-40,83,37,6,6,0,0,"POLYGON ((519902.030 4377005.400, 519911.080 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,4,16,4-16,253,138,11,11,16,1
1,4,17,4-17,214,102,16,16,8,1
2,4,19,4-19,326,95,20,13,15,2
3,4,2,4-2,14,7,4,1,0,0
4,4,20,4-20,255,195,23,9,22,2
...,...,...,...,...,...,...,...,...,...
15,4,37,4-37,327,160,20,19,26,1
16,4,38,4-38,305,115,12,4,13,1
17,4,39,4-39,353,96,6,7,13,1
18,4,40,4-40,83,37,6,6,0,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [21]:
# county 5

# set up
county_num = 5
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

county_df.loc[county_df['precinct']=='1001:DS1, SD1', 'mergeid'] = '1001:DS1,SD1'
county_df.loc[county_df['precinct']=='1001:DS2, SD2', 'mergeid'] = '1001:DS2,SD2'


# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

# by observing the difference in two lists above, deduce the changes required in the dict to properly translate
#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 5 and county name is: Daggett


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,"1001:DS1,SD1",0,22,0,1,0,46,"5-1001:DS1,SD1",5
1,"1001:DS2,SD2",0,6,0,0,0,28,"5-1001:DS2,SD2",5
2,1002,5,28,3,2,0,138,5-1002,5
3,"1002:MS3,SD3",4,9,0,3,0,67,"5-1002:MS3,SD3",5
4,1002:SD2,0,15,3,1,0,54,5-1002:SD2,5
5,1003:SD1,2,0,0,0,0,2,5-1003:SD1,5


6 6 6


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
174,5,"1001:DS1,SD1","5-1001:DS1,SD1",46,22,0,1,0,0,"POLYGON ((664004.500 4540665.600, 664047.300 4..."
175,5,"1001:DS2,SD2","5-1001:DS2,SD2",28,6,0,0,0,0,"POLYGON ((631806.960 4539575.320, 631790.700 4..."
176,5,1002,5-1002,138,28,3,2,5,0,"POLYGON ((613582.290 4538865.490, 613598.690 4..."
177,5,"1002:MS3,SD3","5-1002:MS3,SD3",67,9,0,3,4,0,"POLYGON ((607236.150 4538845.570, 607237.900 4..."
178,5,1002:SD2,5-1002:SD2,54,15,3,1,0,0,"POLYGON ((617398.100 4539485.550, 617396.300 4..."
179,5,1003:SD1,5-1003:SD1,2,0,0,0,2,0,"POLYGON ((664134.200 4536055.300, 664185.600 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,5,"1001:DS1,SD1","5-1001:DS1,SD1",46,22,0,1,0,0
1,5,"1001:DS2,SD2","5-1001:DS2,SD2",28,6,0,0,0,0
2,5,1002,5-1002,138,28,3,2,5,0
3,5,"1002:MS3,SD3","5-1002:MS3,SD3",67,9,0,3,4,0
4,5,1002:SD2,5-1002:SD2,54,15,3,1,0,0
5,5,1003:SD1,5-1003:SD1,2,0,0,0,2,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [21]:
# county 6

# set up
county_num = 6
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(lambda x: x.split(':', 1)[0])
county_df['candidateid'] = county_df['candidate'].map(senator_dict)


# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 6 and county name is: Davis


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,BO01,14,196,11,14,0,434,6-BO01,6
1,BO02,12,166,5,15,0,400,6-BO02,6
2,BO03,9,149,5,12,0,371,6-BO03,6
3,BO04,18,156,3,14,0,420,6-BO04,6
4,BO05,14,104,5,15,0,342,6-BO05,6
...,...,...,...,...,...,...,...,...,...
219,WX03,15,239,18,20,0,503,6-WX03,6
220,WX04,11,124,10,6,0,175,6-WX04,6
221,WX05,6,178,17,20,0,357,6-WX05,6
222,WX06,3,108,4,6,0,247,6-WX06,6


224 228 224


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
180,6,BO01,6-BO01,434,196,11,14,14,0,"POLYGON ((425294.560 4529172.990, 425294.530 4..."
181,6,BO02,6-BO02,400,166,5,15,12,0,"POLYGON ((425957.370 4527420.290, 425955.570 4..."
182,6,BO03,6-BO03,371,149,5,12,9,0,"POLYGON ((427306.650 4528715.770, 427307.810 4..."
183,6,BO04,6-BO04,420,156,3,14,18,0,"POLYGON ((433248.230 4530334.210, 433240.230 4..."
184,6,BO05,6-BO05,342,104,5,15,14,0,"POLYGON ((427302.180 4528005.150, 427302.750 4..."
...,...,...,...,...,...,...,...,...,...,...
403,6,WX07,6-WX07,387,155,12,21,8,0,"POLYGON ((421436.100 4525508.840, 421530.890 4..."
2263,6,DC01,6-DC01,0,0,0,0,0,0,"POLYGON ((402321.480 4549317.100, 402716.900 4..."
2264,6,DC02,6-DC02,0,0,0,0,0,0,"POLYGON ((405054.200 4554496.340, 405072.690 4..."
2265,6,DC03,6-DC03,0,0,0,0,0,0,"POLYGON ((423330.200 4540532.600, 423303.210 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: ['6-DC01' '6-DC02' '6-DC03' '6-DC04']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,6,DC01,6-DC01,0,0,0,0,0,0
1,6,DC02,6-DC02,0,0,0,0,0,0
2,6,DC03,6-DC03,0,0,0,0,0,0
3,6,DC04,6-DC04,0,0,0,0,0,0
4,6,BO01,6-BO01,434,196,11,14,14,0
...,...,...,...,...,...,...,...,...,...
223,6,WX03,6-WX03,503,239,18,20,15,0
224,6,WX04,6-WX04,175,124,10,6,11,0
225,6,WX05,6-WX05,357,178,17,20,6,0
226,6,WX06,6-WX06,247,108,4,6,3,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [22]:
# county 7

# set up
county_num = 7
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df[cleaned_df['uniqueid'] == '7-RO92'] )
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

# by observing the difference in two lists above, deduce the changes required in the dict to properly translate

final_df = cleaned_df

unique_id_dict = {
    '7-RO92': '7-R092'
}

merge_id_dict = {
    'RO92': 'R092'
}

final_df['uniqueid'] = final_df['uniqueid'].replace(unique_id_dict)
final_df['mergeid'] = final_df['mergeid'].replace(merge_id_dict)

final_df = pd.concat([template_df, cleaned_df]).reset_index().drop(columns='index')
display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 7 and county name is: Duchesne


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
31,RO92,4,4,2,2,0,53,7-RO92,7


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AL11,15,12,1,3,0,215,7-AL11,7
1,AL12,1,6,1,0,0,53,7-AL12,7
2,AR11,21,11,0,0,0,67,7-AR11,7
3,BB11,24,16,2,8,1,173,7-BB11,7
4,BB21,4,4,1,2,0,74,7-BB21,7
...,...,...,...,...,...,...,...,...,...
29,RO82,3,18,1,0,0,50,7-RO82,7
30,RO91,0,0,0,0,0,4,7-RO91,7
31,RO92,4,4,2,2,0,53,7-RO92,7
32,TA11,15,16,2,3,0,169,7-TA11,7


34 34 34


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
404,7,AL11,7-AL11,215,12,1,3,15,0,"POLYGON ((563117.920 4520407.000, 563174.420 4..."
405,7,AL12,7-AL12,53,6,1,0,1,0,"POLYGON ((560878.570 4468295.740, 560879.820 4..."
406,7,AR11,7-AR11,67,11,0,0,21,0,"POLYGON ((579363.160 4451801.930, 579362.800 4..."
407,7,BB11,7-BB11,173,16,2,8,24,1,"POLYGON ((565072.380 4519984.380, 565060.670 4..."
408,7,BB21,7-BB21,74,4,1,2,4,0,"POLYGON ((574350.730 4469628.840, 574350.800 4..."
...,...,...,...,...,...,...,...,...,...,...
433,7,RO81,7-RO81,134,13,0,7,13,0,"MULTIPOLYGON (((580836.220 4464060.740, 580837..."
434,7,RO82,7-RO82,50,18,1,0,3,0,"POLYGON ((586967.910 4461063.280, 586317.330 4..."
435,7,RO91,7-RO91,4,0,0,0,0,0,"POLYGON ((583582.420 4461812.580, 583604.090 4..."
436,7,TA11,7-TA11,169,16,2,3,15,0,"POLYGON ((533169.000 4511919.160, 533169.000 4..."




precincts that exist in ut_senators but not in vest 2018  are: ['7-RO92']


precincts that exist in vest 2018 but not in ut senators are: ['7-R092']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,7,AL11,7-AL11,215,12,1,3,15,0
1,7,AL12,7-AL12,53,6,1,0,1,0
2,7,AR11,7-AR11,67,11,0,0,21,0
3,7,BB11,7-BB11,173,16,2,8,24,1
4,7,BB21,7-BB21,74,4,1,2,4,0
...,...,...,...,...,...,...,...,...,...
29,7,RO82,7-RO82,50,18,1,0,3,0
30,7,RO91,7-RO91,4,0,0,0,0,0
31,7,R092,7-R092,53,4,2,2,4,0
32,7,TA11,7-TA11,169,16,2,3,15,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [107]:
# county 6

# typo in vote counts

pd.set_option('display.max_rows', 20)

def strip_id(string):
    split = string.rsplit(' ', 1)
    if len(split) > 1:      # since Canvas is recorded as "Canvas", will not split into list of 2
        return int(split[1])
    else:
        return 13      # Canvas votes are recorded as 13

# set up
county_num = 8
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

display(county_df)
display(county_df[county_df['candidate'] == 'Mitt Romney'])

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

pd.set_option('display.max_rows', 100)


# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)


#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

canvas_votes = final_df[final_df["mergeid"] ==  13]
office_rom = canvas_votes["G18USSRROM"].iloc[0]
office_wil = canvas_votes["G18USSDWIL"].iloc[0]
office_aal = canvas_votes["G18USSCAAL"].iloc[0]

final_df = final_df.sort_values(by = 'G18USSRROM')
for i in range(0, int(office_rom)):
    print(str(int(final_df.iloc[i]['G18USSRROM']) + 1))
    final_df['G18USSRROM'].iloc[i] = str(int(final_df.iloc[i]['G18USSRROM']) + 1)
    print(final_df.iloc[i]['G18USSRROM'])
    
final_df = final_df.sort_values(by = 'G18USSDWIL')
for i in range(0, int(office_wil)):
    final_df['G18USSDWIL'].iloc[i] = str(int(final_df.iloc[i]['G18USSDWIL']) + 1)

final_df = final_df.sort_values(by = 'G18USSCAAL')
for i in range(0, int(office_aal)):
    final_df['G18USSCAAL'].iloc[i] = str(int(final_df.iloc[i]['G18USSCAAL']) + 1)

display(final_df)

print(final_df['G18USSRROM'].apply(int).sum())

concat_dict[county_num] = final_df

county num is: 8 and county name is: Emery


,county,precinct,office,district,candidate,party,votes
40121,Emery,Emery 1,U.S. Senate,NaN,Tim Aalders,Con,5
40122,Emery,Ferron 2,U.S. Senate,NaN,Tim Aalders,Con,7
40123,Emery,Ferron 3,U.S. Senate,NaN,Tim Aalders,Con,7
40124,Emery,Clawson 4,U.S. Senate,NaN,Tim Aalders,Con,5
40125,Emery,Orangeville 5,U.S. Senate,NaN,Tim Aalders,Con,6
...,...,...,...,...,...,...,...
40194,Emery,Huntington 9,U.S. Senate,NaN,write-in,NaN,0
40195,Emery,Cleveland 10,U.S. Senate,NaN,write-in,NaN,0
40196,Emery,Elmo 11,U.S. Senate,NaN,write-in,NaN,0
40197,Emery,Green River 12,U.S. Senate,NaN,write-in,NaN,1


,county,precinct,office,district,candidate,party,votes
40173,Emery,Emery 1,U.S. Senate,NaN,Mitt Romney,Rep,109
40174,Emery,Ferron 2,U.S. Senate,NaN,Mitt Romney,Rep,307
40175,Emery,Ferron 3,U.S. Senate,NaN,Mitt Romney,Rep,234
40176,Emery,Clawson 4,U.S. Senate,NaN,Mitt Romney,Rep,80
40177,Emery,Orangeville 5,U.S. Senate,NaN,Mitt Romney,Rep,261
40178,Emery,Castle Dale 6,U.S. Senate,NaN,Mitt Romney,Rep,222
40179,Emery,Castle Dale 7,U.S. Senate,NaN,Mitt Romney,Rep,268
40180,Emery,Huntington 8,U.S. Senate,NaN,Mitt Romney,Rep,329
40181,Emery,Huntington 9,U.S. Senate,NaN,Mitt Romney,Rep,305
40182,Emery,Cleveland 10,U.S. Senate,NaN,Mitt Romney,Rep,296


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,1,5,16,7,2,0,109,8-1,8
1,2,7,32,16,6,1,307,8-2,8
2,3,7,54,21,6,1,234,8-3,8
3,4,5,12,5,2,0,80,8-4,8
4,5,6,59,21,7,1,261,8-5,8
5,6,5,39,14,4,0,222,8-6,8
6,7,6,45,6,8,1,268,8-7,8
7,8,12,55,20,9,0,329,8-8,8
8,9,8,59,24,8,0,305,8-9,8
9,10,6,43,11,10,0,296,8-10,8


13 13 13


...



precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,8,1,8-1,109,16,7,2,5,0
1,8,2,8-2,307,32,16,6,7,1
2,8,3,8-3,234,54,21,6,7,1
3,8,4,8-4,80,12,5,2,5,0
4,8,5,8-5,261,59,21,7,6,1
5,8,6,8-6,222,39,14,4,5,0
6,8,7,8-7,268,45,6,8,6,1
7,8,8,8-8,329,55,20,9,12,0
8,8,9,8-9,305,59,24,8,8,0
9,8,10,8-10,296,43,11,10,6,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []
110
110
179
179
223
223
235
235
239
239
262
262


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
12,8,13,8-13,6,2,0,0,2,0
7,8,8,8-8,329,55,20,9,12,0
11,8,12,8-12,239,58,10,4,13,1
3,8,4,8-4,80,12,5,2,5,0
0,8,1,8-1,110,16,7,2,5,0
5,8,6,8-6,223,39,14,4,5,0
9,8,10,8-10,296,43,11,10,6,0
6,8,7,8-7,268,45,6,8,6,1
4,8,5,8-5,262,59,21,7,6,1
1,8,2,8-2,307,32,16,6,7,1


2839


In [24]:
# county 9

def strip_id(string):
    split = string.rsplit(' ', 1)
    split2 = split[1].split(':', 1)
    return split2[0]


# set up
county_num = 9
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 9 and county name is: Garfield


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSRROM,uniqueid,CountyID
0,AN1,5,5,0,1,55,9-AN1,9
1,BO2,14,140,0,4,73,9-BO2,9
2,BR11,0,3,0,0,27,9-BR11,9
3,CA3,2,2,1,1,64,9-CA3,9
4,ES4,8,116,9,4,290,9-ES4,9
...,...,...,...,...,...,...,...,...
6,HE6,3,14,0,2,94,9-HE6,9
7,PA7,18,37,3,12,374,9-PA7,9
8,PA8,11,28,4,2,315,9-PA8,9
9,TI9,0,4,0,1,10,9-TI9,9


11 11 11


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
450,9,AN1,9-AN1,55,5,0,1,5,0,"POLYGON ((403168.700 4222982.900, 403564.300 4..."
451,9,BO2,9-BO2,73,140,0,4,14,0,"POLYGON ((464163.400 4222959.500, 464725.600 4..."
452,9,BR11,9-BR11,27,3,0,0,0,0,"POLYGON ((398263.080 4172252.570, 398352.130 4..."
453,9,CA3,9-CA3,64,2,1,1,2,0,"POLYGON ((409151.000 4165082.600, 411282.700 4..."
454,9,ES4,9-ES4,290,116,9,4,8,0,"POLYGON ((432080.100 4207910.300, 432100.100 4..."
...,...,...,...,...,...,...,...,...,...,...
456,9,HE6,9-HE6,94,14,0,2,3,0,"POLYGON ((421187.400 4178547.000, 421205.500 4..."
457,9,PA7,9-PA7,374,37,3,12,18,0,"POLYGON ((387081.800 4223222.200, 387258.700 4..."
458,9,PA8,9-PA8,315,28,4,2,11,0,"POLYGON ((372018.600 4223467.700, 372060.200 4..."
459,9,TI9,9-TI9,10,4,0,1,0,0,"POLYGON ((570016.100 4200064.500, 570002.500 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,9,AN1,9-AN1,55,5,0,1,5,NaN
1,9,BO2,9-BO2,73,140,0,4,14,NaN
2,9,BR11,9-BR11,27,3,0,0,0,NaN
3,9,CA3,9-CA3,64,2,1,1,2,NaN
4,9,ES4,9-ES4,290,116,9,4,8,NaN
...,...,...,...,...,...,...,...,...,...
6,9,HE6,9-HE6,94,14,0,2,3,NaN
7,9,PA7,9-PA7,374,37,3,12,18,NaN
8,9,PA8,9-PA8,315,28,4,2,11,NaN
9,9,TI9,9-TI9,10,4,0,1,0,NaN




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [25]:
# county 10

def strip_id(string):
    split = string.rsplit(' ', 1)
    output = ''
    if len(split[1]) < 5:
        output = '0' + split[1]
        return output
    else:
        return split[1]

# set up
county_num = 10
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators

print(county_df['precinct'].unique())
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)


# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

['01 Moab 1:70' '02 Moab 2:69' '03 Moab 2:70' '04 Moab 3:70'
 '05 Moab 4:69' '06 Moab 4:70' '07 Moab 5:69' '08 Moab 5:70'
 '09 Thompson 6:69' '10 Thompson 6:70' '11 Green River 7:69'
 '12 Green River 7:70' '13 Castle Valley 8:69' '14 Moab 9:69'
 '15 Moab 9:70' '16 Moab 10:70' '17 Moab 11:69' '18 Moab 11:70']
county num is: 10 and county name is: Grand


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSRROM,uniqueid,CountyID
0,01:70,7,200,3,7,100,10-01:70,10
1,02:69,14,404,4,29,397,10-02:69,10
2,02:70,0,18,0,0,0,10-02:70,10
3,03:70,0,81,2,4,95,10-03:70,10
4,04:69,0,0,0,0,0,10-04:69,10
...,...,...,...,...,...,...,...,...
13,09:69,3,41,9,0,59,10-09:69,10
14,09:70,16,206,10,12,183,10-09:70,10
15,10:70,6,195,6,11,196,10-10:70,10
16,11:69,9,178,8,11,251,10-11:69,10


18 18 18


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
461,10,01:70,10-01:70,100,200,3,7,7,0,"POLYGON ((626844.880 4270777.500, 626844.940 4..."
462,10,02:69,10-02:69,397,404,4,29,14,0,"POLYGON ((628559.230 4268594.890, 628572.380 4..."
463,10,02:70,10-02:70,0,18,0,0,0,0,"POLYGON ((628437.380 4269162.500, 628442.810 4..."
464,10,03:70,10-03:70,95,81,2,4,0,0,"POLYGON ((625321.940 4271081.500, 625342.440 4..."
465,10,04:69,10-04:69,0,0,0,0,0,0,"POLYGON ((626781.440 4280780.500, 626856.440 4..."
...,...,...,...,...,...,...,...,...,...,...
474,10,09:69,10-09:69,59,41,9,0,3,0,"POLYGON ((628246.200 4268639.100, 628278.100 4..."
475,10,09:70,10-09:70,183,206,10,12,16,0,"POLYGON ((627497.940 4269532.500, 627509.380 4..."
476,10,10:70,10-10:70,196,195,6,11,6,0,"POLYGON ((626286.310 4269372.500, 626312.250 4..."
477,10,11:69,10-11:69,251,178,8,11,9,0,"POLYGON ((628559.350 4268475.550, 628567.170 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,10,01:70,10-01:70,100,200,3,7,7,NaN
1,10,02:69,10-02:69,397,404,4,29,14,NaN
2,10,02:70,10-02:70,0,18,0,0,0,NaN
3,10,03:70,10-03:70,95,81,2,4,0,NaN
4,10,04:69,10-04:69,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...
13,10,09:69,10-09:69,59,41,9,0,3,NaN
14,10,09:70,10-09:70,183,206,10,12,16,NaN
15,10,10:70,10-10:70,196,195,6,11,6,NaN
16,10,11:69,10-11:69,251,178,8,11,9,NaN




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [26]:
# county 11

# EN0

def strip_id(string):
    output = ''
    if string[:2] == 'CC' or string[:2] == 'CH':
        return (string[:2] + ' ' + str(int(string[2:])))
    if string[:2] == 'EN':
        return (string[:3] + ' ' + string[-1])
    else:
        return string

# set up
county_num = 11
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
display(county_df[county_df['precinct'] == 'IRON UT'])

display(county_name_dict_vest[county_num])
display(county_df)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)


# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')
rows = (final_df[final_df['uniqueid'].isin(missing_precincts)])
display(rows)
final_df.drop(rows.index, inplace = True)
display(final_df)

concat_dict[county_num] = final_df



,county,precinct,office,district,candidate,party,votes
17180,Iron,IRON UT,U.S. Senate,NaN,Tim Aalders,Con,866
17222,Iron,IRON UT,U.S. Senate,NaN,Craig Bowden,Lbt,597
17264,Iron,IRON UT,U.S. Senate,NaN,Reed McCandless,IAP,206
17306,Iron,IRON UT,U.S. Senate,NaN,Jenny Wilson,Dem,2846
17348,Iron,IRON UT,U.S. Senate,NaN,Mitt Romney,Rep,11286
17390,Iron,IRON UT,U.S. Senate,NaN,write-in,NaN,68


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
479,11,BH,11-BH,49,21,1,1,3,0,"POLYGON ((337236.930 4178132.000, 338899.200 4..."
480,11,BRYL,11-BRYL,97,22,5,11,7,0,"POLYGON ((292454.900 4204974.570, 292238.690 4..."
481,11,CC 1,11-CC 1,389,127,4,14,19,2,"POLYGON ((319572.870 4176129.690, 319571.630 4..."
482,11,CC 10,11-CC 10,117,38,2,6,5,0,"POLYGON ((317327.540 4172001.090, 317320.950 4..."
483,11,CC 11,11-CC 11,82,34,1,6,5,0,"POLYGON ((317738.440 4172023.590, 317734.280 4..."
...,...,...,...,...,...,...,...,...,...,...
515,11,PARAGO,11-PARAGO,188,29,6,4,12,0,"POLYGON ((343799.800 4195870.400, 343909.300 4..."
516,11,PARN,11-PARN,498,75,3,14,33,1,"POLYGON ((341391.640 4194302.220, 342286.260 4..."
517,11,PARS,11-PARS,384,88,10,8,44,3,"POLYGON ((344729.470 4192596.190, 344716.400 4..."
518,11,PARV,11-PARV,113,14,1,3,3,0,"POLYGON ((326860.500 4224105.160, 327036.440 4..."


,county,precinct,office,district,candidate,party,votes
17139,Iron,BH,U.S. Senate,NaN,Tim Aalders,Con,3
17140,Iron,CC10,U.S. Senate,NaN,Tim Aalders,Con,5
17141,Iron,CC13,U.S. Senate,NaN,Tim Aalders,Con,9
17142,Iron,CC16,U.S. Senate,NaN,Tim Aalders,Con,44
17143,Iron,CC19,U.S. Senate,NaN,Tim Aalders,Con,30
...,...,...,...,...,...,...,...
17386,Iron,ENO2,U.S. Senate,NaN,write-in,NaN,4
17387,Iron,KANARR,U.S. Senate,NaN,write-in,NaN,3
17388,Iron,NEWCSL,U.S. Senate,NaN,write-in,NaN,2
17389,Iron,PARS,U.S. Senate,NaN,write-in,NaN,3


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,BH,3,21,1,1,0,49,11-BH,11
1,BRYL,7,22,5,11,0,97,11-BRYL,11
2,CC 1,19,127,4,14,2,389,11-CC 1,11
3,CC 10,5,38,2,6,0,117,11-CC 10,11
4,CC 11,5,34,1,6,0,82,11-CC 11,11
...,...,...,...,...,...,...,...,...,...
37,PARAGO,12,29,6,4,0,188,11-PARAGO,11
38,PARN,33,75,3,14,1,498,11-PARN,11
39,PARS,44,88,10,8,3,384,11-PARS,11
40,PARV,3,14,1,3,0,113,11-PARV,11


42 41 42


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
479,11,BH,11-BH,49,21,1,1,3,0,"POLYGON ((337236.930 4178132.000, 338899.200 4..."
480,11,BRYL,11-BRYL,97,22,5,11,7,0,"POLYGON ((292454.900 4204974.570, 292238.690 4..."
481,11,CC 1,11-CC 1,389,127,4,14,19,2,"POLYGON ((319572.870 4176129.690, 319571.630 4..."
482,11,CC 10,11-CC 10,117,38,2,6,5,0,"POLYGON ((317327.540 4172001.090, 317320.950 4..."
483,11,CC 11,11-CC 11,82,34,1,6,5,0,"POLYGON ((317738.440 4172023.590, 317734.280 4..."
...,...,...,...,...,...,...,...,...,...,...
515,11,PARAGO,11-PARAGO,188,29,6,4,12,0,"POLYGON ((343799.800 4195870.400, 343909.300 4..."
516,11,PARN,11-PARN,498,75,3,14,33,1,"POLYGON ((341391.640 4194302.220, 342286.260 4..."
517,11,PARS,11-PARS,384,88,10,8,44,3,"POLYGON ((344729.470 4192596.190, 344716.400 4..."
518,11,PARV,11-PARV,113,14,1,3,3,0,"POLYGON ((326860.500 4224105.160, 327036.440 4..."




precincts that exist in ut_senators but not in vest 2018  are: ['11-IRON UT']


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
32,11,IRON UT,11-IRON UT,11286,2846,206,597,866,68


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,11,BH,11-BH,49,21,1,1,3,0
1,11,BRYL,11-BRYL,97,22,5,11,7,0
2,11,CC 1,11-CC 1,389,127,4,14,19,2
3,11,CC 10,11-CC 10,117,38,2,6,5,0
4,11,CC 11,11-CC 11,82,34,1,6,5,0
...,...,...,...,...,...,...,...,...,...
37,11,PARAGO,11-PARAGO,188,29,6,4,12,0
38,11,PARN,11-PARN,498,75,3,14,33,1
39,11,PARS,11-PARS,384,88,10,8,44,3
40,11,PARV,11-PARV,113,14,1,3,3,0


In [27]:
# county 12


def strip_id(string):
    if string[-2] == '.':
        split = string.rsplit('.', 1)
        output = split[0] + ':' + split[1]
        return output
    return string



# set up
county_num = 12
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)


#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)
concat_dict[county_num] = final_df

county num is: 12 and county name is: Juab


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,CA11,10,5,0,0,0,22,12-CA11,12
1,EU:1,7,21,4,3,0,31,12-EU:1,12
2,EU:2,7,28,0,3,0,59,12-EU:2,12
3,EU:3,5,25,0,1,0,57,12-EU:3,12
4,EU:4,4,16,3,4,0,24,12-EU:4,12
...,...,...,...,...,...,...,...,...,...
14,NE03:2,11,26,8,3,0,252,12-NE03:2,12
15,NE03:3,4,15,3,2,0,99,12-NE03:3,12
16,NE04,13,54,1,5,0,452,12-NE04,12
17,NE05,11,62,5,10,0,339,12-NE05,12


19 20 19


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
520,12,CA11,12-CA11,22,5,0,0,10,0,"POLYGON ((345376.480 4379648.650, 344171.710 4..."
521,12,EU:1,12-EU:1,31,21,4,3,7,0,"POLYGON ((404393.100 4425248.750, 404391.450 4..."
522,12,EU:2,12-EU:2,59,28,0,3,7,0,"POLYGON ((404488.600 4422540.370, 404484.260 4..."
523,12,EU:3,12-EU:3,57,25,0,1,5,0,"POLYGON ((405718.190 4422526.980, 404484.190 4..."
524,12,EU:4,12-EU:4,24,16,3,4,4,0,"POLYGON ((402860.050 4422326.790, 402854.400 4..."
...,...,...,...,...,...,...,...,...,...,...
535,12,NE03:3,12-NE03:3,99,15,3,2,4,0,"POLYGON ((428493.340 4396062.690, 428327.250 4..."
536,12,NE04,12-NE04,452,54,1,5,13,0,"POLYGON ((446684.110 4403949.030, 446695.010 4..."
537,12,NE05,12-NE05,339,62,5,10,11,0,"POLYGON ((428497.490 4395743.400, 428499.060 4..."
538,12,RR13,12-RR13,98,11,3,12,58,0,"POLYGON ((429935.270 4421767.710, 429932.480 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: ['12-NE03:5']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
2268,12,NE03:5,12-NE03:5,0,0,0,0,0,0


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,12,NE03:5,12-NE03:5,0,0,0,0,0,0
1,12,CA11,12-CA11,22,5,0,0,10,0
2,12,EU:1,12-EU:1,31,21,4,3,7,0
3,12,EU:2,12-EU:2,59,28,0,3,7,0
4,12,EU:3,12-EU:3,57,25,0,1,5,0
...,...,...,...,...,...,...,...,...,...
15,12,NE03:2,12-NE03:2,252,26,8,3,11,0
16,12,NE03:3,12-NE03:3,99,15,3,2,4,0
17,12,NE04,12-NE04,452,54,1,5,13,0
18,12,NE05,12-NE05,339,62,5,10,11,0


In [28]:
# county 13

# Do I take out totals here?

def strip_id(string):
    if string[0] == 'K':
        output = string[0] + 'A' + string[1:]
        return output
    return string



# set up
county_num = 13
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)


# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')
rows = (final_df[final_df['uniqueid'].isin(missing_precincts)])
display(rows)
final_df.drop(rows.index, inplace = True)
display(final_df)

concat_dict[county_num] = final_df

county num is: 13 and county name is: Kane


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AL,0,6,2,0,0,55,13-AL,13
1,BU,0,4,1,3,0,6,13-BU,13
2,BW,11,44,5,6,0,84,13-BW,13
3,CM,5,26,0,4,0,116,13-CM,13
4,CW:B,5,6,0,2,0,28,13-CW:B,13
...,...,...,...,...,...,...,...,...,...
8,KA2,11,189,3,17,2,340,13-KA2,13
9,KA3,11,133,4,17,1,483,13-KA3,13
10,KA4,22,220,15,21,2,405,13-KA4,13
11,OR,16,28,4,8,1,226,13-OR,13


13 12 13


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
539,13,AL,13-AL,55,6,2,0,0,0,"POLYGON ((370858.860 4156086.710, 370972.660 4..."
540,13,BU,13-BU,6,4,1,3,0,0,"POLYGON ((442101.710 4105063.140, 442102.510 4..."
541,13,BW,13-BW,84,44,5,6,11,0,"POLYGON ((442101.710 4105063.140, 442088.160 4..."
542,13,CM,13-CM,116,26,0,4,5,0,"POLYGON ((333462.020 4156801.320, 333518.150 4..."
543,13,CW:B,13-CW:B,28,6,0,2,5,0,"POLYGON ((429977.470 4114104.710, 429978.760 4..."
...,...,...,...,...,...,...,...,...,...,...
546,13,KA1,13-KA1,243,70,4,4,8,0,"POLYGON ((413538.260 4109040.650, 413562.330 4..."
547,13,KA2,13-KA2,340,189,3,17,11,2,"POLYGON ((391171.860 4155110.670, 391213.560 4..."
548,13,KA3,13-KA3,483,133,4,17,11,1,"POLYGON ((366598.990 4126832.490, 366634.850 4..."
549,13,KA4,13-KA4,405,220,15,21,22,2,"POLYGON ((362380.960 4100141.040, 362783.850 4..."




precincts that exist in ut_senators but not in vest 2018  are: ['13-TOTALS']


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
13,13,TOTALS,13-TOTALS,2135,746,38,89,109,6


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,12,NE03:5,12-NE03:5,0,0,0,0,0,0
1,13,AL,13-AL,55,6,2,0,0,0
2,13,BU,13-BU,6,4,1,3,0,0
3,13,BW,13-BW,84,44,5,6,11,0
4,13,CM,13-CM,116,26,0,4,5,0
...,...,...,...,...,...,...,...,...,...
8,13,KA1,13-KA1,243,70,4,4,8,0
9,13,KA2,13-KA2,340,189,3,17,11,2
10,13,KA3,13-KA3,483,133,4,17,11,1
11,13,KA4,13-KA4,405,220,15,21,22,2


In [29]:
# county 14

# set up
county_num = 14
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

precinct_list = list(county_df['precinct'].unique())
vest_list = list(county_name_dict_vest[county_num]['mergeid'])
trans_dict = dict(zip(precinct_list, vest_list))

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].map(trans_dict)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)


# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 14 and county name is: Millard


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,ABR14,7,1,3,2,2,44,14-ABR14,14
1,DEL17,43,74,7,13,0,465,14-DEL17,14
2,DEL18,46,90,8,17,2,538,14-DEL18,14
3,DO5,17,21,4,13,3,200,14-DO5,14
4,FIL1,12,54,4,6,1,485,14-FIL1,14
...,...,...,...,...,...,...,...,...,...
12,LYN19,5,8,1,0,0,42,14-LYN19,14
13,MEA3,10,16,2,4,0,130,14-MEA3,14
14,OAK6,31,24,10,6,2,268,14-OAK6,14
15,SCIP9,8,27,0,0,0,139,14-SCIP9,14


17 17 17


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
551,14,ABR14,14-ABR14,44,1,3,2,7,2,"POLYGON ((287556.280 4381006.920, 289535.550 4..."
552,14,DEL17,14-DEL17,465,74,7,13,43,0,"POLYGON ((374128.300 4365910.850, 373870.980 4..."
553,14,DEL18,14-DEL18,538,90,8,17,46,2,"POLYGON ((366612.180 4357068.230, 366612.050 4..."
554,14,DO5,14-DO5,200,21,4,13,17,3,"POLYGON ((359967.160 4353104.600, 360776.650 4..."
555,14,FIL1,14-FIL1,485,54,4,6,12,1,"POLYGON ((401982.780 4314077.020, 401922.700 4..."
...,...,...,...,...,...,...,...,...,...,...
563,14,LYN19,14-LYN19,42,8,1,0,5,0,"POLYGON ((385657.350 4374006.700, 385602.400 4..."
564,14,MEA3,14-MEA3,130,16,2,4,10,0,"POLYGON ((379270.470 4308885.620, 380920.010 4..."
565,14,OAK6,14-OAK6,268,24,10,6,31,2,"POLYGON ((397309.680 4366854.810, 397210.910 4..."
566,14,SCIP9,14-SCIP9,139,27,0,0,8,0,"POLYGON ((403961.630 4353935.630, 404363.000 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,14,ABR14,14-ABR14,44,1,3,2,7,2
1,14,DEL17,14-DEL17,465,74,7,13,43,0
2,14,DEL18,14-DEL18,538,90,8,17,46,2
3,14,DO5,14-DO5,200,21,4,13,17,3
4,14,FIL1,14-FIL1,485,54,4,6,12,1
...,...,...,...,...,...,...,...,...,...
12,14,LYN19,14-LYN19,42,8,1,0,5,0
13,14,MEA3,14-MEA3,130,16,2,4,10,0
14,14,OAK6,14-OAK6,268,24,10,6,31,2
15,14,SCIP9,14-SCIP9,139,27,0,0,8,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [30]:
# county 15

def strip_id(string):
    split = string.rsplit(' ', 1)[1]
    split2 = split.split('.')
    output = split2[0] + ':' + split2[1]
    return output

# set up
county_num = 15
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)


missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 15 and county name is: Morgan


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,10:4,11,43,0,18,1,319,15-10:4,15
1,11:4,9,37,4,8,0,148,15-11:4,15
2,12:4,9,45,3,9,0,301,15-12:4,15
3,13:5,15,39,8,6,3,444,15-13:5,15
4,14:5,14,45,7,5,0,304,15-14:5,15
...,...,...,...,...,...,...,...,...,...
10,5:2,8,34,9,9,0,187,15-5:2,15
11,6:2,17,40,2,9,3,219,15-6:2,15
12,7:3,10,20,2,3,1,232,15-7:3,15
13,8:3,6,23,0,7,2,226,15-8:3,15


15 15 15


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
568,15,1:1,15-1:1,307,79,5,10,20,3,"POLYGON ((431569.630 4561834.170, 431550.130 4..."
569,15,10:4,15-10:4,319,43,0,18,11,1,"POLYGON ((440762.270 4544845.210, 440793.230 4..."
570,15,11:4,15-11:4,148,37,4,8,9,0,"POLYGON ((443328.250 4542698.210, 443346.490 4..."
571,15,12:4,15-12:4,301,45,3,9,9,0,"POLYGON ((443346.490 4542648.920, 443444.380 4..."
572,15,13:5,15-13:5,444,39,8,6,15,3,"POLYGON ((445082.810 4542456.070, 444933.350 4..."
...,...,...,...,...,...,...,...,...,...,...
578,15,5:2,15-5:2,187,34,9,9,8,0,"POLYGON ((430923.690 4554530.100, 430987.830 4..."
579,15,6:2,15-6:2,219,40,2,9,17,3,"POLYGON ((442341.700 4551052.980, 442342.830 4..."
580,15,7:3,15-7:3,232,20,2,3,10,1,"POLYGON ((437434.820 4549057.620, 437436.870 4..."
581,15,8:3,15-8:3,226,23,0,7,6,2,"POLYGON ((441943.890 4542511.550, 441945.480 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,15,10:4,15-10:4,319,43,0,18,11,1
1,15,11:4,15-11:4,148,37,4,8,9,0
2,15,12:4,15-12:4,301,45,3,9,9,0
3,15,13:5,15-13:5,444,39,8,6,15,3
4,15,14:5,15-14:5,304,45,7,5,14,0
...,...,...,...,...,...,...,...,...,...
10,15,5:2,15-5:2,187,34,9,9,8,0
11,15,6:2,15-6:2,219,40,2,9,17,3
12,15,7:3,15-7:3,232,20,2,3,10,1
13,15,8:3,15-8:3,226,23,0,7,6,2




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [31]:
# county 16

#2:3 vs 2:2

def strip_id(string):
    split = string.rsplit(' ', 1)[1]
    if len(split) > 1:
        split2 = split.split('-', 1)
        output = split2[0] + ':' + split2[1]
        return output
    elif split == '4' or split == '5':
        output = split + ':' + '4'
        return output
    elif split == '2':
        output = split + ':' + '2'
        return output
    else:
        output = split + ':' + '3'
        return output



# set up
county_num = 16
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(strip_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 16 and county name is: Piute


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
40277,Piute,Marysvale 1-1,U.S. Senate,NaN,Tim Aalders,Constitution,24,1:1,G18USSCAAL
40278,Piute,Marysvale 1-1,U.S. Senate,NaN,Craig R. Bowden,Libertarian,11,1:1,G18USSLBOW
40279,Piute,Marysvale 1-1,U.S. Senate,NaN,Reed C. McCandless,IAP,3,1:1,G18USSIMCC
40280,Piute,Marysvale 1-1,U.S. Senate,NaN,Jenny Wilson,Democrat,22,1:1,G18USSDWIL
40281,Piute,Marysvale 1-1,U.S. Senate,NaN,Mitt Romney,Republican,118,1:1,G18USSRROM
...,...,...,...,...,...,...,...,...,...
40344,Piute,Greenwich 5,U.S. Senate,NaN,Craig R. Bowden,Libertarian,0,5:4,G18USSLBOW
40345,Piute,Greenwich 5,U.S. Senate,NaN,Reed C. McCandless,IAP,0,5:4,G18USSIMCC
40346,Piute,Greenwich 5,U.S. Senate,NaN,Jenny Wilson,Democrat,9,5:4,G18USSDWIL
40347,Piute,Greenwich 5,U.S. Senate,NaN,Mitt Romney,Republican,38,5:4,G18USSRROM


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,1:1,24,22,3,11,1,118,16-1:1,16
1,1:2,1,8,0,1,0,47,16-1:2,16
2,2:2,2,13,3,4,0,72,16-2:2,16
3,3:3,6,11,1,2,0,85,16-3:3,16
4,3:5,16,4,1,3,0,82,16-3:5,16
5,4:4,12,10,0,0,1,82,16-4:4,16
6,5:4,1,9,0,0,0,38,16-5:4,16


7 7 7


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
583,16,1:1,16-1:1,118,22,3,11,24,1,"POLYGON ((367601.000 4263500.900, 368241.840 4..."
584,16,1:2,16-1:2,47,8,0,1,1,0,"POLYGON ((390098.630 4263376.610, 390151.670 4..."
585,16,2:2,16-2:2,72,13,3,4,2,0,"POLYGON ((402409.000 4238964.000, 402339.000 4..."
586,16,3:3,16-3:3,85,11,1,2,6,0,"POLYGON ((377985.200 4229578.800, 378018.200 4..."
587,16,3:5,16-3:5,82,4,1,3,16,0,"MULTIPOLYGON (((393673.500 4228223.100, 393673..."
588,16,4:4,16-4:4,82,10,0,0,12,1,"POLYGON ((407271.000 4241971.000, 411716.000 4..."
589,16,5:4,16-5:4,38,9,0,0,1,0,"POLYGON ((408266.600 4262024.600, 409164.600 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [32]:
# county 17

# set up
county_num = 17
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)


#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([cleaned_df, missing_row]).reset_index().drop(columns='index')

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)



display(final_df)
concat_dict[county_num] = final_df

county num is: 17 and county name is: Rich


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
41584,Rich,1,U.S. Senate,NaN,Tim Aalders,Con,6,1,G18USSCAAL
41585,Rich,2,U.S. Senate,NaN,Tim Aalders,Con,9,2,G18USSCAAL
41586,Rich,3,U.S. Senate,NaN,Tim Aalders,Con,5,3,G18USSCAAL
41587,Rich,4,U.S. Senate,NaN,Tim Aalders,Con,8,4,G18USSCAAL
41588,Rich,5,U.S. Senate,NaN,Tim Aalders,Con,5,5,G18USSCAAL
...,...,...,...,...,...,...,...,...,...
41609,Rich,1,U.S. Senate,NaN,Write-ins,NaN,0,1,G18USSOWRI
41610,Rich,2,U.S. Senate,NaN,Write-ins,NaN,0,2,G18USSOWRI
41611,Rich,3,U.S. Senate,NaN,Write-ins,NaN,0,3,G18USSOWRI
41612,Rich,4,U.S. Senate,NaN,Write-ins,NaN,0,4,G18USSOWRI


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,1,6,15,0,4,0,159,17-1,17
1,2,9,24,1,1,0,207,17-2,17
2,3,5,21,2,12,0,257,17-3,17
3,4,8,30,1,2,0,159,17-4,17
4,5,5,34,1,5,0,129,17-5,17


5 6 5


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
590,17,1,17-1,159,15,0,4,6,0,"POLYGON ((469222.200 4614315.900, 469251.400 4..."
591,17,2,17-2,207,24,1,1,9,0,"POLYGON ((483412.950 4612976.350, 483455.970 4..."
592,17,3,17-3,257,21,2,12,5,0,"POLYGON ((496111.700 4644391.700, 496125.000 4..."
593,17,4,17-4,159,30,1,2,8,0,"POLYGON ((476855.700 4641589.600, 476890.500 4..."
594,17,5,17-5,129,34,1,5,5,0,"POLYGON ((467354.710 4642409.010, 467269.730 4..."
2269,17,UNINH,17-UNINH,0,0,0,0,0,0,"POLYGON ((476910.860 4643876.500, 476898.550 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: ['17-UNINH']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
2269,17,UNINH,17-UNINH,0,0,0,0,0,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,1,6,15,0,4,0,159,17-1,17
1,2,9,24,1,1,0,207,17-2,17
2,3,5,21,2,12,0,257,17-3,17
3,4,8,30,1,2,0,159,17-4,17
4,5,5,34,1,5,0,129,17-5,17
5,UNINH,0,0,0,0,0,0,17-UNINH,17


In [33]:
# county 18

# set up
county_num = 18
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([cleaned_df, missing_row]).reset_index().drop(columns='index')

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

display(final_df)
concat_dict[county_num] = final_df

county num is: 18 and county name is: Salt Lake


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
77,Salt Lake,ALT001,U.S. Senate,NaN,MITT ROMNEY,REP,24,ALT001,G18USSRROM
78,Salt Lake,BLF001,U.S. Senate,NaN,MITT ROMNEY,REP,735,BLF001,G18USSRROM
79,Salt Lake,BLF003,U.S. Senate,NaN,MITT ROMNEY,REP,687,BLF003,G18USSRROM
80,Salt Lake,BLF004,U.S. Senate,NaN,MITT ROMNEY,REP,696,BLF004,G18USSRROM
81,Salt Lake,BLF005,U.S. Senate,NaN,MITT ROMNEY,REP,15,BLF005,G18USSRROM
...,...,...,...,...,...,...,...,...,...
10238,Salt Lake,WVC074,U.S. Senate,NaN,ABE KORB,NaN,0,WVC074,G18USSOWRI
10239,Salt Lake,WVC076,U.S. Senate,NaN,ABE KORB,NaN,0,WVC076,G18USSOWRI
10240,Salt Lake,WVC077,U.S. Senate,NaN,ABE KORB,NaN,0,WVC077,G18USSOWRI
10241,Salt Lake,WVC078,U.S. Senate,NaN,ABE KORB,NaN,0,WVC078,G18USSOWRI


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,ALT001,1,99,1,3,0,24,18-ALT001,18
1,BLF001,26,148,13,29,0,735,18-BLF001,18
2,BLF003,26,232,8,22,3,687,18-BLF003,18
3,BLF004,23,128,4,20,0,696,18-BLF004,18
4,BLF005,3,4,0,0,0,15,18-BLF005,18
...,...,...,...,...,...,...,...,...,...
777,WVC074,3,93,5,7,0,114,18-WVC074,18
778,WVC076,0,35,0,4,0,58,18-WVC076,18
779,WVC077,4,184,4,14,0,139,18-WVC077,18
780,WVC078,3,107,5,17,0,150,18-WVC078,18


782 837 782


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
595,18,ALT001,18-ALT001,24,99,1,3,1,0,"POLYGON ((447709.960 4494899.800, 447737.330 4..."
596,18,BLF001,18-BLF001,735,148,13,29,26,0,"POLYGON ((417236.870 4483768.860, 419204.190 4..."
597,18,BLF003,18-BLF003,687,232,8,22,26,3,"POLYGON ((421262.720 4483724.930, 421261.850 4..."
598,18,BLF004,18-BLF004,696,128,4,20,23,0,"POLYGON ((419632.660 4482528.440, 419600.830 4..."
599,18,BLF005,18-BLF005,15,4,0,0,3,0,"POLYGON ((422048.120 4483316.950, 422048.080 4..."
...,...,...,...,...,...,...,...,...,...,...
2320,18,NRV069,18-NRV069,0,0,0,0,0,0,"POLYGON ((427542.050 4496177.060, 427538.290 4..."
2321,18,NRV071,18-NRV071,0,0,0,0,0,0,"POLYGON ((427563.660 4495772.200, 427531.790 4..."
2322,18,SAN088,18-SAN088,0,0,0,0,0,0,"POLYGON ((426191.040 4491382.400, 426318.420 4..."
2323,18,SLC154,18-SLC154,0,0,0,0,0,0,"POLYGON ((428876.800 4507890.150, 428846.060 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: ['18-DRP005' '18-NLC001' '18-NRV001' '18-NRV002' '18-NRV003' '18-NRV004'
 '18-NRV005' '18-NRV006' '18-NRV007' '18-NRV008' '18-NRV009' '18-NRV010'
 '18-NRV011' '18-NRV013' '18-NRV014' '18-NRV015' '18-NRV016' '18-NRV017'
 '18-NRV018' '18-NRV019' '18-NRV020' '18-NRV022' '18-NRV024' '18-NRV025'
 '18-NRV026' '18-NRV028' '18-NRV029' '18-NRV030' '18-NRV031' '18-NRV033'
 '18-NRV034' '18-NRV035' '18-NRV037' '18-NRV040' '18-NRV045' '18-NRV046'
 '18-NRV047' '18-NRV050' '18-NRV051' '18-NRV052' '18-NRV053' '18-NRV054'
 '18-NRV055' '18-NRV056' '18-NRV057' '18-NRV060' '18-NRV061' '18-NRV062'
 '18-NRV063' '18-NRV067' '18-NRV069' '18-NRV071' '18-SAN088' '18-SLC154'
 '18-SNB002']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
2270,18,DRP005,18-DRP005,0,0,0,0,0,0
2271,18,NLC001,18-NLC001,0,0,0,0,0,0
2272,18,NRV001,18-NRV001,0,0,0,0,0,0
2273,18,NRV002,18-NRV002,0,0,0,0,0,0
2274,18,NRV003,18-NRV003,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2320,18,NRV069,18-NRV069,0,0,0,0,0,0
2321,18,NRV071,18-NRV071,0,0,0,0,0,0
2322,18,SAN088,18-SAN088,0,0,0,0,0,0
2323,18,SLC154,18-SLC154,0,0,0,0,0,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,ALT001,1,99,1,3,0,24,18-ALT001,18
1,BLF001,26,148,13,29,0,735,18-BLF001,18
2,BLF003,26,232,8,22,3,687,18-BLF003,18
3,BLF004,23,128,4,20,0,696,18-BLF004,18
4,BLF005,3,4,0,0,0,15,18-BLF005,18
...,...,...,...,...,...,...,...,...,...
832,NRV069,0,0,0,0,0,0,18-NRV069,18
833,NRV071,0,0,0,0,0,0,18-NRV071,18
834,SAN088,0,0,0,0,0,0,18-SAN088,18
835,SLC154,0,0,0,0,0,0,18-SLC154,18


In [34]:
# county 19

def split_id(string):
    return string.rsplit(' ', 1)[1]

# set up
county_num = 19
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(split_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df


county num is: 19 and county name is: San Juan


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
24819,San Juan,Precinct 1,U.S. Senate,NaN,Tim Aalders,CON,13,1,G18USSCAAL
24820,San Juan,Precinct 1,U.S. Senate,NaN,Craig R. Bowden,LIB,13,1,G18USSLBOW
24821,San Juan,Precinct 1,U.S. Senate,NaN,Reed C. McCandless,IAP,3,1,G18USSIMCC
24822,San Juan,Precinct 1,U.S. Senate,NaN,Jenny Wilson,DEM,54,1,G18USSDWIL
24823,San Juan,Precinct 1,U.S. Senate,NaN,Mitt Romney,REP,123,1,G18USSRROM
...,...,...,...,...,...,...,...,...,...
24952,San Juan,Precinct 12,U.S. Senate,NaN,Craig R. Bowden,LIB,4,12,G18USSLBOW
24953,San Juan,Precinct 12,U.S. Senate,NaN,Reed C. McCandless,IAP,2,12,G18USSIMCC
24954,San Juan,Precinct 12,U.S. Senate,NaN,Jenny Wilson,DEM,40,12,G18USSDWIL
24955,San Juan,Precinct 12,U.S. Senate,NaN,Mitt Romney,REP,246,12,G18USSRROM


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,1,13,54,3,13,1,123,19-1,19
1,10,0,171,2,7,0,48,19-10,19
2,11,14,727,24,34,2,224,19-11,19
3,12,8,40,2,4,1,246,19-12,19
4,2,38,179,19,11,1,822,19-2,19
...,...,...,...,...,...,...,...,...,...
7,5,17,45,8,11,2,327,19-5,19
8,6,1,8,1,0,0,66,19-6,19
9,7,7,47,4,8,0,319,19-7,19
10,8,15,257,8,9,0,274,19-8,19


12 12 12


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1377,19,1,19-1,123,54,3,13,13,1,"POLYGON ((634603.550 4259748.380, 634496.240 4..."
1378,19,10,19-10,48,171,2,7,0,0,"POLYGON ((627890.440 4132495.540, 627886.020 4..."
1379,19,11,19-11,224,727,24,34,14,2,"POLYGON ((601392.200 4230228.470, 601560.530 4..."
1380,19,12,19-12,246,40,2,4,8,1,"POLYGON ((634311.560 4164898.680, 634309.230 4..."
1381,19,2,19-2,822,179,19,11,38,1,"POLYGON ((669204.300 4261457.790, 669213.100 4..."
...,...,...,...,...,...,...,...,...,...,...
1384,19,5,19-5,327,45,8,11,17,2,"POLYGON ((633859.360 4167227.520, 633909.700 4..."
1385,19,6,19-6,66,8,1,0,1,0,"POLYGON ((633741.850 4171744.080, 633746.880 4..."
1386,19,7,19-7,319,47,4,8,7,0,"POLYGON ((632816.490 4168855.730, 632874.770 4..."
1387,19,8,19-8,274,257,8,9,15,0,"POLYGON ((643263.480 4172049.530, 643264.920 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [35]:
# county 20

def split_id(string):
    return string.split(' ', 1)[0]

# set up
county_num = 20
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(split_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df


county num is: 20 and county name is: Sanpete


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
22894,Sanpete,AXTSSD Axtell SSD,U.S. Senate,NaN,Tim Aalders,Con,12,AXTSSD,G18USSCAAL
22895,Sanpete,AXTU Axtell,U.S. Senate,NaN,Tim Aalders,Con,0,AXTU,G18USSCAAL
22896,Sanpete,CENM Centerfield City,U.S. Senate,NaN,Tim Aalders,Con,24,CENM,G18USSCAAL
22897,Sanpete,EPH1M Ephraim City 1 (North),U.S. Senate,NaN,Tim Aalders,Con,27,EPH1M,G18USSCAAL
22898,Sanpete,EPH2M Ephraim City 2 (South W),U.S. Senate,NaN,Tim Aalders,Con,19,EPH2M,G18USSCAAL
...,...,...,...,...,...,...,...,...,...
23063,Sanpete,SPRM Spring City,U.S. Senate,NaN,write-ins,NaN,0,SPRM,G18USSOWRI
23064,Sanpete,SPRU Spring City Unicorporated,U.S. Senate,NaN,write-ins,NaN,1,SPRU,G18USSOWRI
23065,Sanpete,"SSU Centerfield, Sterling, Fayette,Gunnison Un",U.S. Senate,NaN,write-ins,NaN,0,SSU,G18USSOWRI
23066,Sanpete,STRM Sterling Town,U.S. Senate,NaN,write-ins,NaN,0,STRM,G18USSOWRI


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AXTSSD,12,12,3,1,0,65,20-AXTSSD,20
1,AXTU,0,1,0,0,0,27,20-AXTU,20
2,CENM,24,33,6,4,2,350,20-CENM,20
3,EPH1M,27,83,11,10,1,404,20-EPH1M,20
4,EPH2M,19,65,5,10,1,355,20-EPH2M,20
...,...,...,...,...,...,...,...,...,...
24,SPRM,22,101,3,1,0,326,20-SPRM,20
25,SPRU,12,14,5,1,1,90,20-SPRU,20
26,SSU,5,24,4,2,0,138,20-SSU,20
27,STRM,10,8,1,1,0,82,20-STRM,20


29 29 29


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1389,20,AXTSSD,20-AXTSSD,65,12,3,1,12,0,"POLYGON ((431078.760 4324434.670, 431085.810 4..."
1390,20,AXTU,20-AXTU,27,1,0,0,0,0,"POLYGON ((419492.300 4329594.300, 419529.400 4..."
1391,20,CENM,20-CENM,350,33,6,4,24,2,"POLYGON ((430308.060 4330060.540, 429226.500 4..."
1392,20,EPH1M,20-EPH1M,404,83,11,10,27,1,"POLYGON ((449467.060 4358655.460, 449465.580 4..."
1393,20,EPH2M,20-EPH2M,355,65,5,10,19,1,"POLYGON ((450076.480 4355765.420, 450096.400 4..."
...,...,...,...,...,...,...,...,...,...,...
1413,20,SPRM,20-SPRM,326,101,3,1,22,0,"POLYGON ((458436.890 4371393.720, 458478.480 4..."
1414,20,SPRU,20-SPRU,90,14,5,1,12,1,"POLYGON ((467713.650 4373095.570, 467751.900 4..."
1415,20,SSU,20-SSU,138,24,4,2,5,0,"POLYGON ((419455.700 4329600.300, 419353.200 4..."
1416,20,STRM,20-STRM,82,8,1,1,10,0,"POLYGON ((440948.680 4338958.020, 440950.040 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [36]:
# county 21

def split_id(string):
    output = string.rsplit(' ', 1)
    if len(output) > 1:
        return output[1]
    return string

# set up
county_num = 21
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(split_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

final_df = pd.concat([template_df, cleaned_df]).reset_index().drop(columns='index')
rows = (final_df[final_df['uniqueid'].isin(missing_precincts)])
display(rows)
final_df.drop(rows.index, inplace = True)
display(final_df)

concat_dict[county_num] = final_df


county num is: 21 and county name is: Sevier


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
23238,Sevier,Annabella 1,U.S. Senate,NaN,Tim Aalders,Constitution,17,1,G18USSCAAL
23239,Sevier,Annabella 1,U.S. Senate,NaN,Craig R. Bowden,Libertarian,12,1,G18USSLBOW
23240,Sevier,Annabella 1,U.S. Senate,NaN,Reed C. McCandless,IAP,3,1,G18USSIMCC
23241,Sevier,Annabella 1,U.S. Senate,NaN,Jenny Wilson,Democrat,25,1,G18USSDWIL
23242,Sevier,Annabella 1,U.S. Senate,NaN,Mitt Romney,Republican,293,1,G18USSRROM
...,...,...,...,...,...,...,...,...,...
23603,Sevier,TOTALS,U.S. Senate,NaN,Craig R. Bowden,Libertarian,173,TOTALS,G18USSLBOW
23604,Sevier,TOTALS,U.S. Senate,NaN,Reed C. McCandless,IAP,113,TOTALS,G18USSIMCC
23605,Sevier,TOTALS,U.S. Senate,NaN,Jenny Wilson,Democrat,771,TOTALS,G18USSDWIL
23606,Sevier,TOTALS,U.S. Senate,NaN,Mitt Romney,Republican,5810,TOTALS,G18USSRROM


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,1,17,25,3,12,2,293,21-1,21
1,10,9,28,3,8,1,249,21-10,21
2,11,16,37,4,11,4,177,21-11,21
3,11.2,3,2,2,0,0,57,21-11.2,21
4,12,11,33,5,11,7,259,21-12,21
...,...,...,...,...,...,...,...,...,...
24,7.1.3,17,18,2,4,0,112,21-7.1.3,21
25,7.2,7,8,3,0,0,57,21-7.2,21
26,8,35,69,11,19,4,503,21-8,21
27,9,36,80,8,16,2,432,21-9,21


29 28 29


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1418,21,1,21-1,293,25,3,12,17,2,"POLYGON ((409090.890 4288577.030, 409101.830 4..."
1419,21,10,21-10,249,28,3,8,9,1,"POLYGON ((414703.450 4322295.500, 414710.540 4..."
1420,21,11,21-11,177,37,4,11,16,4,"POLYGON ((406926.160 4292049.390, 406927.570 4..."
1421,21,11.2,21-11.2,57,2,2,0,3,0,"POLYGON ((410113.410 4293601.810, 410114.340 4..."
1422,21,12,21-12,259,33,5,11,11,7,"POLYGON ((405093.740 4292386.950, 405104.680 4..."
...,...,...,...,...,...,...,...,...,...,...
1441,21,6,21-6,232,20,3,4,8,1,"POLYGON ((416764.040 4295214.710, 416787.490 4..."
1442,21,7.1.3,21-7.1.3,112,18,2,4,17,0,"POLYGON ((397204.550 4278977.580, 397217.960 4..."
1443,21,7.2,21-7.2,57,8,3,0,7,0,"POLYGON ((397267.370 4279423.830, 397171.800 4..."
1444,21,8,21-8,503,69,11,19,35,4,"POLYGON ((402398.170 4277552.730, 402217.180 4..."




precincts that exist in ut_senators but not in vest 2018  are: ['21-TOTALS']


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
28,21,TOTALS,21-TOTALS,5810,771,113,173,408,50


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,21,1,21-1,293,25,3,12,17,2
1,21,10,21-10,249,28,3,8,9,1
2,21,11,21-11,177,37,4,11,16,4
3,21,11.2,21-11.2,57,2,2,0,3,0
4,21,12,21-12,259,33,5,11,11,7
...,...,...,...,...,...,...,...,...,...
23,21,6,21-6,232,20,3,4,8,1
24,21,7.1.3,21-7.1.3,112,18,2,4,17,0
25,21,7.2,21-7.2,57,8,3,0,7,0
26,21,8,21-8,503,69,11,19,35,4


In [60]:
# county 22

# set up
county_num = 22
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators

#create dict
county_dict_keys = county_df['precinct'].unique()
county_dict_values = county_name_dict_vest[county_num]['mergeid'].unique()
county_dict = dict(zip(county_dict_keys, county_dict_values))

county_vest_2016 = ut_vest_2016[ut_vest_2016['CountyID'] == county_num]
county_name_to_id_dict = county_vest_2016.set_index('precinct').to_dict()['PrecinctID'] #dict based of ut vest 2016 precincts
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].map(county_dict)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([missing_row, cleaned_df]).reset_index().drop(columns='index')

display(final_df)
concat_dict[county_num] = final_df

county num is: 22 and county name is: Summit


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
39030,Summit,Bitner,U.S. Senate,NaN,Tim Aalders,Con,9,BTN34,G18USSCAAL
39031,Summit,Chalk Creek,U.S. Senate,NaN,Tim Aalders,Con,3,CCRK20,G18USSCAAL
39032,Summit,Coalville,U.S. Senate,NaN,Tim Aalders,Con,23,COAL19,G18USSCAAL
39033,Summit,Deer Valley North,U.S. Senate,NaN,Tim Aalders,Con,1,DVN1,G18USSCAAL
39034,Summit,Deer Valley South,U.S. Senate,NaN,Tim Aalders,Con,10,DVS31,G18USSCAAL
...,...,...,...,...,...,...,...,...,...
39331,Summit,Wanship 17:10,U.S. Senate,NaN,write-in,NaN,0,WAN17:10,G18USSOWRI
39332,Summit,Wanship 17:2,U.S. Senate,NaN,write-in,NaN,0,WAN17:2,G18USSOWRI
39333,Summit,Wanship 17:32,U.S. Senate,NaN,write-in,NaN,0,WAN17:32,G18USSOWRI
39334,Summit,Weber Canyon 21:20,U.S. Senate,NaN,write-in,NaN,0,WEBC21:20,G18USSOWRI


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,BTN34,9,383,3,17,0,232,22-BTN34,22
1,CCRK20,3,31,1,6,2,149,22-CCRK20,22
2,COAL19,23,136,4,16,1,338,22-COAL19,22
3,DVN1,1,174,1,5,0,111,22-DVN1,22
4,DVS31,10,337,2,4,2,311,22-DVS31,22
...,...,...,...,...,...,...,...,...,...
46,WAN17:10,8,159,4,9,0,215,22-WAN17:10,22
47,WAN17:2,3,133,2,3,0,67,22-WAN17:2,22
48,WAN17:32,0,15,0,0,0,16,22-WAN17:32,22
49,WEBC21:20,7,57,5,3,0,86,22-WEBC21:20,22


51 51 51


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1446,22,BTN34,22-BTN34,232,383,3,17,9,0,"POLYGON ((456823.270 4513282.010, 456841.320 4..."
1447,22,CCRK20,22-CCRK20,149,31,1,6,3,2,"POLYGON ((482605.410 4543057.230, 482613.010 4..."
1448,22,COAL19,22-COAL19,338,136,4,16,23,1,"POLYGON ((467782.050 4529131.890, 467780.830 4..."
1449,22,DVN1,22-DVN1,111,174,1,5,1,0,"POLYGON ((460786.670 4502657.590, 460795.770 4..."
1450,22,DVS31,22-DVS31,311,337,2,4,10,2,"POLYGON ((463135.670 4502673.200, 463166.560 4..."
...,...,...,...,...,...,...,...,...,...,...
1492,22,WAN17:10,22-WAN17:10,215,159,4,9,8,0,"POLYGON ((468614.480 4521944.940, 468612.920 4..."
1493,22,WAN17:2,22-WAN17:2,67,133,2,3,3,0,"POLYGON ((460239.500 4521407.000, 460244.000 4..."
1494,22,WAN17:32,22-WAN17:32,16,15,0,0,0,0,"POLYGON ((463315.600 4510094.140, 462639.200 4..."
1495,22,WEBC21:20,22-WEBC21:20,86,57,5,3,7,0,"POLYGON ((502063.590 4493615.380, 502063.500 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,22,BTN34,22-BTN34,232,383,3,17,9,0
1,22,CCRK20,22-CCRK20,149,31,1,6,3,2
2,22,COAL19,22-COAL19,338,136,4,16,23,1
3,22,DVN1,22-DVN1,111,174,1,5,1,0
4,22,DVS31,22-DVS31,311,337,2,4,10,2
...,...,...,...,...,...,...,...,...,...
46,22,WAN17:10,22-WAN17:10,215,159,4,9,8,0
47,22,WAN17:2,22-WAN17:2,67,133,2,3,3,0
48,22,WAN17:32,22-WAN17:32,16,15,0,0,0,0
49,22,WEBC21:20,22-WEBC21:20,86,57,5,3,7,0


In [61]:
# county 23

# set up
county_num = 23
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 23 and county name is: Tooele


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
24963,Tooele,DUG-01,U.S. Senate,NaN,Tim Aalders,CON,3,DUG-01,G18USSCAAL
24964,Tooele,DUG-01,U.S. Senate,NaN,Craig R. Bowden,LIB,3,DUG-01,G18USSLBOW
24965,Tooele,DUG-01,U.S. Senate,NaN,Reed C. McCandless,IAP,0,DUG-01,G18USSIMCC
24966,Tooele,DUG-01,U.S. Senate,NaN,Jenny Wilson,DEM,11,DUG-01,G18USSDWIL
24967,Tooele,DUG-01,U.S. Senate,NaN,Mitt Romney,REP,34,DUG-01,G18USSRROM
...,...,...,...,...,...,...,...,...,...
26103,Tooele,WEND,U.S. Senate,NaN,Tim Aalders,CON,2,WEND,G18USSCAAL
26104,Tooele,WEND,U.S. Senate,NaN,Craig R. Bowden,LIB,4,WEND,G18USSLBOW
26105,Tooele,WEND,U.S. Senate,NaN,Reed C. McCandless,IAP,0,WEND,G18USSIMCC
26106,Tooele,WEND,U.S. Senate,NaN,Jenny Wilson,DEM,21,WEND,G18USSDWIL


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSRROM,uniqueid,CountyID
0,DUG-01,3,11,0,3,34,23-DUG-01,23
1,DUG-02,4,11,0,3,15,23-DUG-02,23
2,ERD-01:1,27,130,11,22,522,23-ERD-01:1,23
3,ERD-01:2,0,4,0,0,17,23-ERD-01:2,23
4,ERD-02:1,24,45,7,20,179,23-ERD-02:1,23
...,...,...,...,...,...,...,...,...
53,T-20,22,132,3,20,330,23-T-20,23
54,T-21,41,219,11,30,408,23-T-21,23
55,TERRA,2,5,1,4,24,23-TERRA,23
56,VRN-01:1-01:2,8,15,2,1,94,23-VRN-01:1-01:2,23


58 58 58


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1497,23,DUG-01,23-DUG-01,34,11,0,3,3,0,"POLYGON ((353384.850 4454921.030, 353065.240 4..."
1498,23,DUG-02,23-DUG-02,15,11,0,3,4,0,"POLYGON ((350497.860 4456356.610, 351409.790 4..."
1499,23,ERD-01:1,23-ERD-01:1,522,130,11,22,27,0,"POLYGON ((384053.910 4494057.990, 384046.960 4..."
1500,23,ERD-01:2,23-ERD-01:2,17,4,0,0,0,0,"POLYGON ((390482.430 4497035.910, 390460.750 4..."
1501,23,ERD-02:1,23-ERD-02:1,179,45,7,20,24,0,"POLYGON ((393691.990 4495315.940, 393704.750 4..."
...,...,...,...,...,...,...,...,...,...,...
1550,23,T-20,23-T-20,330,132,3,20,22,0,"POLYGON ((388646.040 4485693.850, 388647.490 4..."
1551,23,T-21,23-T-21,408,219,11,30,41,0,"MULTIPOLYGON (((389084.240 4485692.350, 389119..."
1552,23,TERRA,23-TERRA,24,5,1,4,2,0,"POLYGON ((307834.590 4422335.370, 307840.110 4..."
1553,23,VRN-01:1-01:2,23-VRN-01:1-01:2,94,15,2,1,8,0,"POLYGON ((381483.710 4457594.040, 381505.390 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [75]:
# county 24

# set up
county_num = 24
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators

#rename vernal city precinct:

county_df['precinct'] = np.where(county_df.precinct == 'Vernal City #8', 'Vernal City #08', county_df.precinct)
county_df['precinct'] = np.where(county_df.precinct == 'Vernal City #9', 'Vernal City #09', county_df.precinct)

# create dict
county_dict_keys = sorted(list(county_df['precinct'].unique()))
county_dict_values = county_name_dict_vest[county_num]['mergeid'].unique()
county_dict = dict(zip(county_dict_keys, county_dict_values))

print(county_dict)

print('county num is:', county_num, 'and county name is:', county_name)


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].map(county_dict)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num
pd.set_option('display.max_rows', 500)


# display and compare results
display(cleaned_df[cleaned_df['mergeid'] == 'VC09'])
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)


{'Ashley East #15': 'AE15', 'Ashley West #22': 'AW16', 'Avalon Willows #16': 'AW22', 'Ballard #19': 'BL19', 'Davis East #2': 'DE02', 'Davis North #24': 'DN24', 'Davis West #3': 'DW03', 'Ft. Duchesne #18': 'FD18', 'Glines North #23': 'GN23', 'Glines South #17': 'GS17', 'Jensen #1': 'JN01', 'Lapoint/Tridell #21': 'LT21', 'Maeser East #6': 'MD07', 'Maeser West #25': 'ME06', 'Maeser/Dry Fork #7': 'MW25', 'Naples #4': 'NP04', 'South #5': 'SO05', 'Southwest 26': 'SW26', 'Vernal City #08': 'VC08', 'Vernal City #09': 'VC09', 'Vernal City #10': 'VC10', 'Vernal City #11': 'VC11', 'Vernal City #12': 'VC12', 'Vernal City #13': 'VC13', 'Vernal City #14': 'VC14', 'Whiterocks #20': 'WR20'}
county num is: 24 and county name is: Uintah


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
39731,Uintah,Jensen #1,U.S. Senate,NaN,Jenny Wilson,Dem,47,JN01,G18USSDWIL
39732,Uintah,Davis East #2,U.S. Senate,NaN,Jenny Wilson,Dem,20,DE02,G18USSDWIL
39733,Uintah,Davis West #3,U.S. Senate,NaN,Jenny Wilson,Dem,28,DW03,G18USSDWIL
39734,Uintah,Naples #4,U.S. Senate,NaN,Jenny Wilson,Dem,71,NP04,G18USSDWIL
39735,Uintah,Davis North #24,U.S. Senate,NaN,Jenny Wilson,Dem,12,DN24,G18USSDWIL
39736,Uintah,Vernal City #11,U.S. Senate,NaN,Jenny Wilson,Dem,41,VC11,G18USSDWIL
39737,Uintah,Vernal City #12,U.S. Senate,NaN,Jenny Wilson,Dem,39,VC12,G18USSDWIL
39738,Uintah,Vernal City #14,U.S. Senate,NaN,Jenny Wilson,Dem,39,VC14,G18USSDWIL
39739,Uintah,Ashley East #15,U.S. Senate,NaN,Jenny Wilson,Dem,36,AE15,G18USSDWIL
39740,Uintah,Ashley West #22,U.S. Senate,NaN,Jenny Wilson,Dem,33,AW16,G18USSDWIL


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
19,VC09,13,50,7,12,0,242,24-VC09,24


26 26 26


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1555,24,AE15,24-AE15,382,36,9,13,26,1,"POLYGON ((638258.410 4517577.160, 639656.850 4..."
1556,24,AW16,24-AW16,84,22,1,5,8,0,"MULTIPOLYGON (((620025.690 4452967.890, 620013..."
1557,24,AW22,24-AW22,328,33,9,12,32,0,"POLYGON ((623735.060 4489483.030, 623745.060 4..."
1558,24,BL19,24-BL19,319,65,8,6,41,4,"POLYGON ((592239.120 4466738.450, 592352.640 4..."
1559,24,DE02,24-DE02,302,20,8,9,17,1,"POLYGON ((629215.400 4473828.730, 629244.620 4..."
1560,24,DN24,24-DN24,190,12,5,12,23,2,"POLYGON ((629322.140 4476047.820, 629330.450 4..."
1561,24,DW03,24-DW03,283,28,6,11,42,2,"POLYGON ((626046.140 4474539.020, 626050.950 4..."
1562,24,FD18,24-FD18,119,83,7,6,23,7,"POLYGON ((596764.970 4473184.330, 596802.100 4..."
1563,24,GN23,24-GN23,335,45,2,13,24,2,"POLYGON ((617477.810 4479586.210, 617479.930 4..."
1564,24,GS17,24-GS17,313,36,4,11,15,1,"POLYGON ((622376.140 4477892.080, 622430.000 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [40]:
# county 25


# set up
county_num = 25
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
display(ut_geo[ut_geo['VistaID'] == 'AF11'])
print('county num is:', county_num, 'and county name is:', county_name)


display(county_df[county_df['precinct'] == 'AF11'])


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([cleaned_df, missing_row]).reset_index().drop(columns='index')

rows = (final_df[final_df['uniqueid'].isin(missing_precincts)])
display(rows)
final_df.drop(rows.index, inplace = True)
display(final_df)

display(final_df)

missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df


,OBJECTID,CountyID,VistaID,PrecinctID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area,geometry
1183,1184,25,AF11,AF11,None,2021.1,2021-03-02,None,AndreaB,2021-03-02,{8435A71D-1E5B-4D83-8472-F6AE509AE54A},0.038345,0.000073,"POLYGON ((-111.80901 40.38311, -111.80902 40.3..."


county num is: 25 and county name is: Utah


,county,precinct,office,district,candidate,party,votes
17861,Utah,AF11,U.S. Senate,NaN,Tim Aalders,Con,15
17890,Utah,AF11,U.S. Senate,NaN,Craig Bowden,Lbt,17
17919,Utah,AF11,U.S. Senate,NaN,Reed McCandless,IAP,6
17948,Utah,AF11,U.S. Senate,NaN,Jenny Wilson,Dem,98
17977,Utah,AF11,U.S. Senate,NaN,Mitt Romney,Rep,466


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
17842,Utah,AF01,U.S. Senate,NaN,Tim Aalders,Con,23,AF01,G18USSCAAL
17843,Utah,AF01:UN,U.S. Senate,NaN,Tim Aalders,Con,2,AF01:UN,G18USSCAAL
17844,Utah,AF01S,U.S. Senate,NaN,Tim Aalders,Con,0,AF01S,G18USSCAAL
17845,Utah,AF02,U.S. Senate,NaN,Tim Aalders,Con,16,AF02,G18USSCAAL
17846,Utah,AF03,U.S. Senate,NaN,Tim Aalders,Con,19,AF03,G18USSCAAL
...,...,...,...,...,...,...,...,...,...
19698,Utah,VI01S,U.S. Senate,NaN,Mitt Romney,Rep,33,VI01S,G18USSRROM
19699,Utah,VI01S2,U.S. Senate,NaN,Mitt Romney,Rep,20,VI01S2,G18USSRROM
19700,Utah,VI01UL,U.S. Senate,NaN,Mitt Romney,Rep,0,VI01UL,G18USSRROM
19701,Utah,VI02,U.S. Senate,NaN,Mitt Romney,Rep,124,VI02,G18USSRROM


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AF01,23,173,4,24,0,863,25-AF01,25
1,AF01:UN,2,26,1,3,0,55,25-AF01:UN,25
2,AF01S,0,0,0,0,0,0,25-AF01S,25
3,AF02,16,126,10,19,0,402,25-AF02,25
4,AF03,19,95,8,9,0,674,25-AF03,25
...,...,...,...,...,...,...,...,...,...
367,VI01S,1,11,0,0,0,33,25-VI01S,25
368,VI01S2,1,1,0,0,0,20,25-VI01S2,25
369,VI01UL,0,0,0,0,0,0,25-VI01UL,25
370,VI02,7,110,7,9,0,124,25-VI02,25


...

,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
2325,25,SR09:UN,25-SR09:UN,0,0,0,0,0,0
2326,25,SR10:UN,25-SR10:UN,0,0,0,0,0,0


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
20,AF11:UN,0,0,0,0,0,0,25-AF11:UN,25
38,CH02:UN,0,0,0,0,0,0,25-CH02:UN,25
111,LE18:UN,0,0,0,0,0,0,25-LE18:UN,25
137,MA05:UN,0,0,0,0,0,0,25-MA05:UN,25
318,SP09:UN,0,0,0,0,0,0,25-SP09:UN,25
324,SP14:UN,0,0,0,0,0,0,25-SP14:UN,25
347,SR08:UN,0,0,0,0,0,0,25-SR08:UN,25


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AF01,23,173,4,24,0,863,25-AF01,25
1,AF01:UN,2,26,1,3,0,55,25-AF01:UN,25
2,AF01S,0,0,0,0,0,0,25-AF01S,25
3,AF02,16,126,10,19,0,402,25-AF02,25
4,AF03,19,95,8,9,0,674,25-AF03,25
...,...,...,...,...,...,...,...,...,...
369,VI01UL,0,0,0,0,0,0,25-VI01UL,25
370,VI02,7,110,7,9,0,124,25-VI02,25
371,WH01,33,72,4,15,0,511,25-WH01,25
372,SR09:UN,0,0,0,0,0,0,25-SR09:UN,25


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,AF01,23,173,4,24,0,863,25-AF01,25
1,AF01:UN,2,26,1,3,0,55,25-AF01:UN,25
2,AF01S,0,0,0,0,0,0,25-AF01S,25
3,AF02,16,126,10,19,0,402,25-AF02,25
4,AF03,19,95,8,9,0,674,25-AF03,25
...,...,...,...,...,...,...,...,...,...
369,VI01UL,0,0,0,0,0,0,25-VI01UL,25
370,VI02,7,110,7,9,0,124,25-VI02,25
371,WH01,33,72,4,15,0,511,25-WH01,25
372,SR09:UN,0,0,0,0,0,0,25-SR09:UN,25




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [41]:
# county 26


def split_id(string):
    output = string.rsplit(' ', 1)
    if len(output) > 1:
        return output[1]
    return string

# set up
county_num = 26
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].apply(split_id)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df

county num is: 26 and county name is: Wasatch


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
16584,Wasatch,Wasatch 10,U.S. Senate,NaN,Jenny Wilson,Dem,149,10,G18USSDWIL
16585,Wasatch,Wasatch 11,U.S. Senate,NaN,Jenny Wilson,Dem,113,11,G18USSDWIL
16586,Wasatch,Wasatch 12,U.S. Senate,NaN,Jenny Wilson,Dem,97,12,G18USSDWIL
16587,Wasatch,Wasatch 13,U.S. Senate,NaN,Jenny Wilson,Dem,79,13,G18USSDWIL
16588,Wasatch,Wasatch 14,U.S. Senate,NaN,Jenny Wilson,Dem,167,14,G18USSDWIL
...,...,...,...,...,...,...,...,...,...
16813,Wasatch,Wasatch 54:1,U.S. Senate,NaN,write-in,NaN,0,54:1,G18USSOWRI
16814,Wasatch,Wasatch 54:2,U.S. Senate,NaN,write-in,NaN,0,54:2,G18USSOWRI
16815,Wasatch,Wasatch 54:3,U.S. Senate,NaN,write-in,NaN,1,54:3,G18USSOWRI
16816,Wasatch,Wasatch 55,U.S. Senate,NaN,write-in,NaN,0,55,G18USSOWRI


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,10,3,149,6,4,0,132,26-10,26
1,11,10,113,3,7,0,143,26-11,26
2,12,4,97,4,6,1,167,26-12,26
3,13,6,79,7,12,1,215,26-13,26
4,14,6,167,4,10,5,571,26-14,26
...,...,...,...,...,...,...,...,...,...
34,54:1,1,1,0,0,0,2,26-54:1,26
35,54:2,0,80,0,0,0,42,26-54:2,26
36,54:3,2,42,1,0,1,134,26-54:3,26
37,55,3,8,0,0,0,24,26-55,26


39 39 39


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1946,26,10,26-10,132,149,4,6,3,0,"POLYGON ((465052.920 4485171.390, 465020.190 4..."
1947,26,11,26-11,143,113,3,7,10,0,"POLYGON ((463965.230 4484934.140, 463988.100 4..."
1948,26,12,26-12,167,97,4,6,4,1,"POLYGON ((464375.730 4484054.140, 464396.400 4..."
1949,26,13,26-13,215,79,7,12,6,1,"POLYGON ((464998.420 4484184.400, 465036.880 4..."
1950,26,14,26-14,571,167,4,10,6,5,"POLYGON ((466426.000 4482529.580, 466374.460 4..."
...,...,...,...,...,...,...,...,...,...,...
1980,26,54:1,26-54:1,2,1,0,0,1,0,"MULTIPOLYGON (((459319.040 4497087.200, 458822..."
1981,26,54:2,26-54:2,42,80,0,0,0,0,"POLYGON ((460512.880 4487808.910, 460535.340 4..."
1982,26,54:3,26-54:3,134,42,1,0,2,1,"MULTIPOLYGON (((459291.100 4482788.140, 459307..."
1983,26,55,26-55,24,8,0,0,3,0,"MULTIPOLYGON (((461451.860 4482215.380, 461460..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [42]:
# county 27


def split_id(string):
    output = string.rsplit(' ', 1)
    if len(output) > 1:
        return output[1]
    return string

# set up
county_num = 27
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)


#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([cleaned_df, missing_row]).reset_index().drop(columns='index')

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)



display(final_df)
concat_dict[county_num] = final_df

concat_dict.update({county_num : final_df})


county num is: 27 and county name is: Washington


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
40354,Washington,AV51,U.S. Senate,NaN,Tim Aalders,Con,37,AV51,G18USSCAAL
40355,Washington,COAND,U.S. Senate,NaN,Tim Aalders,Con,0,COAND,G18USSCAAL
40356,Washington,COCDSR,U.S. Senate,NaN,Tim Aalders,Con,3,COCDSR,G18USSCAAL
40357,Washington,COCNBR,U.S. Senate,NaN,Tim Aalders,Con,19,COCNBR,G18USSCAAL
40358,Washington,CODAV,U.S. Senate,NaN,Tim Aalders,Con,40,CODAV,G18USSCAAL
...,...,...,...,...,...,...,...,...,...
40904,Washington,WA69,U.S. Senate,NaN,Mitt Romney,Rep,511,WA69,G18USSRROM
40905,Washington,WA70,U.S. Senate,NaN,Mitt Romney,Rep,454,WA70,G18USSRROM
40906,Washington,WA71,U.S. Senate,NaN,Mitt Romney,Rep,600,WA71,G18USSRROM
40907,Washington,WA72,U.S. Senate,NaN,Mitt Romney,Rep,343,WA72,G18USSRROM


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSRROM,uniqueid,CountyID
0,AV51,37,66,20,12,175,27-AV51,27
1,COAND,0,0,0,0,0,27-COAND,27
2,COCDSR,3,17,1,1,102,27-COCDSR,27
3,COCNBR,19,18,6,8,134,27-COCNBR,27
4,CODAV,40,118,7,14,307,27-CODAV,27
...,...,...,...,...,...,...,...,...
106,WA69,35,95,9,15,511,27-WA69,27
107,WA70,23,62,5,15,454,27-WA70,27
108,WA71,59,161,4,23,600,27-WA71,27
109,WA72,27,56,6,13,343,27-WA72,27


111 119 111


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
1985,27,AV51,27-AV51,175,66,20,12,37,0,"POLYGON ((309430.450 4112702.480, 312879.450 4..."
1986,27,COAND,27-COAND,0,0,0,0,0,0,"POLYGON ((278311.800 4166456.420, 278344.640 4..."
1987,27,COCDSR,27-COCDSR,102,17,1,1,3,0,"POLYGON ((295169.970 4110212.440, 295159.200 4..."
1988,27,COCNBR,27-COCNBR,134,18,6,8,19,0,"POLYGON ((282503.230 4144335.550, 282719.270 4..."
1989,27,CODAV,27-CODAV,307,118,7,14,40,0,"POLYGON ((281445.090 4133735.920, 281492.930 4..."
...,...,...,...,...,...,...,...,...,...,...
2330,27,COUN04,27-COUN04,0,0,0,0,0,0,"POLYGON ((283305.460 4122754.700, 283305.070 4..."
2331,27,COUN05,27-COUN05,0,0,0,0,0,0,"POLYGON ((310223.210 4149598.960, 310191.590 4..."
2332,27,COUN06,27-COUN06,0,0,0,0,0,0,"POLYGON ((255057.190 4154374.220, 256665.370 4..."
2333,27,COUN07,27-COUN07,0,0,0,0,0,0,"POLYGON ((284912.790 4113292.470, 284905.170 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: ['27-COUN01' '27-COUN02' '27-COUN03' '27-COUN04' '27-COUN05' '27-COUN06'
 '27-COUN07' '27-SGAIR']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
2327,27,COUN01,27-COUN01,0,0,0,0,0,0
2328,27,COUN02,27-COUN02,0,0,0,0,0,0
2329,27,COUN03,27-COUN03,0,0,0,0,0,0
2330,27,COUN04,27-COUN04,0,0,0,0,0,0
2331,27,COUN05,27-COUN05,0,0,0,0,0,0
2332,27,COUN06,27-COUN06,0,0,0,0,0,0
2333,27,COUN07,27-COUN07,0,0,0,0,0,0
2334,27,SGAIR,27-SGAIR,0,0,0,0,0,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSRROM,uniqueid,CountyID,G18USSOWRI
0,AV51,37,66,20,12,175,27-AV51,27,NaN
1,COAND,0,0,0,0,0,27-COAND,27,NaN
2,COCDSR,3,17,1,1,102,27-COCDSR,27,NaN
3,COCNBR,19,18,6,8,134,27-COCNBR,27,NaN
4,CODAV,40,118,7,14,307,27-CODAV,27,NaN
...,...,...,...,...,...,...,...,...,...
114,COUN04,0,0,0,0,0,27-COUN04,27,0.0
115,COUN05,0,0,0,0,0,27-COUN05,27,0.0
116,COUN06,0,0,0,0,0,27-COUN06,27,0.0
117,COUN07,0,0,0,0,0,27-COUN07,27,0.0


In [43]:
# county 28


def split_id(string):
    output = string.rsplit(' ', 1)
    if len(output) > 1:
        return output[1]
    return string

# set up
county_num = 28
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)


precinct_list = list(county_df['precinct'].unique())
vest_list = list(county_name_dict_vest[county_num]['mergeid'])
trans_dict = dict(zip(precinct_list, vest_list))


# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct'].map(trans_dict)
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

concat_dict[county_num] = final_df


county num is: 28 and county name is: Wayne


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
0,Wayne,Fremont,U.S. Senate,NaN,Tim Aalders,Con,7,BI04,G18USSCAAL
1,Wayne,Loa in/out,U.S. Senate,NaN,Tim Aalders,Con,6,FR01,G18USSCAAL
2,Wayne,Lyman in/out,U.S. Senate,NaN,Tim Aalders,Con,6,HA07,G18USSCAAL
3,Wayne,Bicknell in/out,U.S. Senate,NaN,Tim Aalders,Con,8,LO02,G18USSCAAL
4,Wayne,Teesdale,U.S. Senate,NaN,Tim Aalders,Con,3,LY03,G18USSCAAL
...,...,...,...,...,...,...,...,...,...
37,Wayne,Lyman in/out,U.S. Senate,NaN,write-in,NaN,0,HA07,G18USSOWRI
38,Wayne,Bicknell in/out,U.S. Senate,NaN,write-in,NaN,0,LO02,G18USSOWRI
39,Wayne,Teesdale,U.S. Senate,NaN,write-in,NaN,0,LY03,G18USSOWRI
40,Wayne,Torrey in/out,U.S. Senate,NaN,write-in,NaN,2,TE05,G18USSOWRI


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,BI04,7,11,1,0,0,105,28-BI04,28
1,FR01,6,16,0,3,4,284,28-FR01,28
2,HA07,6,8,4,1,0,124,28-HA07,28
3,LO02,8,20,4,7,0,186,28-LO02,28
4,LY03,3,81,5,4,0,102,28-LY03,28
5,TE05,4,116,3,8,2,112,28-TE05,28
6,TO06,14,15,0,2,2,85,28-TO06,28


7 7 7


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
2096,28,BI04,28-BI04,186,20,4,7,8,0,"POLYGON ((444316.200 4246769.850, 444351.120 4..."
2097,28,FR01,28-FR01,105,11,1,0,7,0,"POLYGON ((449482.410 4256309.010, 449473.910 4..."
2098,28,HA07,28-HA07,86,15,0,2,14,2,"POLYGON ((585361.980 4261340.440, 585407.210 4..."
2099,28,LO02,28-LO02,284,16,0,3,6,4,"POLYGON ((436942.300 4254036.710, 438052.890 4..."
2100,28,LY03,28-LY03,124,8,4,1,6,0,"POLYGON ((452699.990 4255394.690, 452692.580 4..."
2101,28,TE05,28-TE05,102,81,5,4,3,0,"POLYGON ((481380.760 4222795.510, 480615.190 4..."
2102,28,TO06,28-TO06,112,116,3,8,4,2,"POLYGON ((484709.970 4256279.620, 484715.030 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


In [44]:
# county 29


def split_id(string):
    output = string.rsplit(' ', 1)
    if len(output) > 1:
        return output[1]
    return string

# set up
county_num = 29
county_name = county_num_to_name[county_num]
county_df = county_name_dict[county_name].copy() #curr county dataframe ut_senators
print('county num is:', county_num, 'and county name is:', county_name)

# add additional fields based on precinct and senator dict
county_df['mergeid'] = county_df['precinct']
county_df['candidateid'] = county_df['candidate'].map(senator_dict)

display(county_df)
#replace(*->0)
# pivot table 
cleaned_df = county_df.pivot_table(index='mergeid', columns='candidateid' , values='votes', aggfunc='first', fill_value=0).reset_index()
cleaned_df['uniqueid'] = (str(county_num) + '-' + cleaned_df['mergeid'].astype(str))
cleaned_df['CountyID'] = county_num

# display and compare results
display(cleaned_df)
print(len(cleaned_df), len(county_name_dict_vest[county_num]), len(county_df['precinct'].unique()))
display(county_name_dict_vest[county_num])

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(cleaned_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(cleaned_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

#add missing rows
vest18 = county_name_dict_vest[county_num]
missing_row = vest18[vest18['uniqueid'].isin(missing_precincts2)] #the missing entry in raw election data
missing_row = pd.DataFrame(missing_row.drop(columns='geometry'))
display(missing_row)
final_df = pd.concat([cleaned_df, missing_row]).reset_index().drop(columns='index')

# finding which precincts are not in ut_senators
missing_precincts = np.setdiff1d(list(final_df['uniqueid']),list(county_name_dict_vest[county_num]['uniqueid'])) #list of precincts from ut vest 2016 with no matches
missing_precincts2 = np.setdiff1d(list(county_name_dict_vest[county_num]['uniqueid']), list(final_df['uniqueid'])) #list of precincts from ut vest 2016 with no matches
print('\n\nprecincts that exist in ut_senators but not in vest 2018  are:', missing_precincts)
print('\n\nprecincts that exist in vest 2018 but not in ut senators are:', missing_precincts2)

def sum_vote(num):
    if num == '*':
        return 0
    else:
        return int(num)

# WHY ASTERISK
display(final_df)
print(final_df['G18USSDWIL'].apply(sum_vote).sum())
print(vest18['G18USSRROM'].apply(sum_vote).sum())


concat_dict[county_num] = final_df

county num is: 29 and county name is: Weber


,county,precinct,office,district,candidate,party,votes,mergeid,candidateid
27760,Weber,CYN001,U.S. Senate,NaN,Jenny Wilson,Dem,53,CYN001,G18USSDWIL
27761,Weber,FRW001,U.S. Senate,NaN,Jenny Wilson,Dem,79,FRW001,G18USSDWIL
27762,Weber,FRW002,U.S. Senate,NaN,Jenny Wilson,Dem,132,FRW002,G18USSDWIL
27763,Weber,FRW003,U.S. Senate,NaN,Jenny Wilson,Dem,129,FRW003,G18USSDWIL
27764,Weber,FRW004,U.S. Senate,NaN,Jenny Wilson,Dem,143,FRW004,G18USSDWIL
...,...,...,...,...,...,...,...,...,...
28703,Weber,WHV007,U.S. Senate,NaN,write-in,NaN,3,WHV007,G18USSOWRI
28704,Weber,WHV008,U.S. Senate,NaN,write-in,NaN,1,WHV008,G18USSOWRI
28705,Weber,WNO001,U.S. Senate,NaN,write-in,NaN,*,WNO001,G18USSOWRI
28706,Weber,WNO002,U.S. Senate,NaN,write-in,NaN,*,WNO002,G18USSOWRI


candidateid,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,CYN001,3,53,2,6,0,37,29-CYN001,29
1,FRW001,18,79,3,11,1,476,29-FRW001,29
2,FRW002,24,132,5,12,1,304,29-FRW002,29
3,FRW003,24,129,11,29,3,583,29-FRW003,29
4,FRW004,32,143,12,35,2,652,29-FRW004,29
...,...,...,...,...,...,...,...,...,...
153,WHV007,20,96,3,11,3,336,29-WHV007,29
154,WHV008,10,46,6,9,1,90,29-WHV008,29
155,WNO001,*,8,*,*,*,*,29-WNO001,29
156,WNO002,*,*,*,*,*,*,29-WNO002,29


158 160 158


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
2103,29,CYN001,29-CYN001,37,53,2,6,3,0,"POLYGON ((429767.380 4567616.890, 429752.330 4..."
2104,29,FRW001,29-FRW001,476,79,3,11,18,1,"POLYGON ((415532.900 4571511.100, 415555.020 4..."
2105,29,FRW002,29-FRW002,304,132,5,12,24,1,"POLYGON ((414104.540 4571410.010, 414104.310 4..."
2106,29,FRW003,29-FRW003,583,129,11,29,24,3,"POLYGON ((412691.000 4573709.660, 412690.930 4..."
2107,29,FRW004,29-FRW004,652,143,12,35,32,2,"POLYGON ((413427.540 4576634.630, 413436.480 4..."
...,...,...,...,...,...,...,...,...,...,...
2258,29,WNO001,29-WNO001,1,0,0,0,0,0,"MULTIPOLYGON (((418865.700 4571506.440, 418865..."
2259,29,WNO002,29-WNO002,7,1,0,1,2,0,"MULTIPOLYGON (((420125.050 4572851.460, 420130..."
2260,29,WNO003,29-WNO003,24,17,0,0,0,0,"MULTIPOLYGON (((419849.820 4571401.230, 419849..."
2335,29,LAKE,29-LAKE,0,0,0,0,0,0,"POLYGON ((431658.370 4571461.460, 431647.750 4..."




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: ['29-LAKE' '29-UNKN']


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
2335,29,LAKE,29-LAKE,0,0,0,0,0,0
2336,29,UNKN,29-UNKN,0,0,0,0,0,0




precincts that exist in ut_senators but not in vest 2018  are: []


precincts that exist in vest 2018 but not in ut senators are: []


,mergeid,G18USSCAAL,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSOWRI,G18USSRROM,uniqueid,CountyID
0,CYN001,3,53,2,6,0,37,29-CYN001,29
1,FRW001,18,79,3,11,1,476,29-FRW001,29
2,FRW002,24,132,5,12,1,304,29-FRW002,29
3,FRW003,24,129,11,29,3,583,29-FRW003,29
4,FRW004,32,143,12,35,2,652,29-FRW004,29
...,...,...,...,...,...,...,...,...,...
155,WNO001,*,8,*,*,*,*,29-WNO001,29
156,WNO002,*,*,*,*,*,*,29-WNO002,29
157,WNO003,0,17,0,0,0,24,29-WNO003,29
158,LAKE,0,0,0,0,0,0,29-LAKE,29


24509
49458


In [105]:
## df = pd.concat(list(concat_dict.values()), ignore_index=True)

list_of_counties = []
for k,v in concat_dict.items():
        v = v.reset_index()
        list_of_counties.append(v)

        
df = pd.concat(list_of_counties, ignore_index=True)
display(concat_dict[27].duplicated())

display(df[df['CountyID'] == 27])

display(df)
display(ut_vest)
print(len(df.uniqueid.unique()))
print(len(ut_vest.uniqueid.unique()))

dupl = df[df.duplicated()]
display(dupl)
print(dupl.CountyID.unique())

display(df)
df = df.drop_duplicates()
display(df)

0      False
1      False
2      False
3      False
4      False
       ...  
114    False
115    False
116    False
117    False
118    False
Length: 119, dtype: bool

,index,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
3988,0,27,AV51,27-AV51,175,66,20,12,37,NaN
3989,1,27,COAND,27-COAND,0,0,0,0,0,NaN
3990,2,27,COCDSR,27-COCDSR,102,17,1,1,3,NaN
3991,3,27,COCNBR,27-COCNBR,134,18,6,8,19,NaN
3992,4,27,CODAV,27-CODAV,307,118,7,14,40,NaN
...,...,...,...,...,...,...,...,...,...,...
4221,114,27,COUN04,27-COUN04,0,0,0,0,0,0.0
4222,115,27,COUN05,27-COUN05,0,0,0,0,0,0.0
4223,116,27,COUN06,27-COUN06,0,0,0,0,0,0.0
4224,117,27,COUN07,27-COUN07,0,0,0,0,0,0.0


,index,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,0,1,BV01,1-BV01,402,46,8,9,19,1
1,1,1,BV02,1-BV02,310,46,3,10,17,0
2,2,1,BV03,1-BV03,244,30,0,2,12,2
3,3,1,BV04,1-BV04,84,7,1,4,23,0
4,4,1,GR01,1-GR01,56,9,6,1,1,1
...,...,...,...,...,...,...,...,...,...,...
4381,155,29,WNO001,29-WNO001,*,8,*,*,*,*
4382,156,29,WNO002,29-WNO002,*,*,*,*,*,*
4383,157,29,WNO003,29-WNO003,24,17,0,0,0,0
4384,158,29,LAKE,29-LAKE,0,0,0,0,0,0


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
0,1,BV01,1-BV01,402,46,8,9,19,1,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,1-BV02,310,46,3,10,17,0,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,1-BV03,244,30,0,2,12,2,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,1-BV04,84,7,1,4,23,0,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,1-GR01,56,9,6,1,1,1,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
...,...,...,...,...,...,...,...,...,...,...
2332,27,COUN06,27-COUN06,0,0,0,0,0,0,"POLYGON ((255057.190 4154374.220, 256665.370 4..."
2333,27,COUN07,27-COUN07,0,0,0,0,0,0,"POLYGON ((284912.790 4113292.470, 284905.170 4..."
2334,27,SGAIR,27-SGAIR,0,0,0,0,0,0,"POLYGON ((277885.930 4102780.480, 277884.100 4..."
2335,29,LAKE,29-LAKE,0,0,0,0,0,0,"POLYGON ((431658.370 4571461.460, 431647.750 4..."


2159
2337


,index,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
547,0,12,NE03:5,12-NE03:5,0,0,0,0,0,0
592,0,15,10:4,15-10:4,319,43,0,18,11,1
593,1,15,11:4,15-11:4,148,37,4,8,9,0
594,2,15,12:4,15-12:4,301,45,3,9,9,0
595,3,15,13:5,15-13:5,444,39,8,6,15,3
...,...,...,...,...,...,...,...,...,...,...
4221,114,27,COUN04,27-COUN04,0,0,0,0,0,0.0
4222,115,27,COUN05,27-COUN05,0,0,0,0,0,0.0
4223,116,27,COUN06,27-COUN06,0,0,0,0,0,0.0
4224,117,27,COUN07,27-COUN07,0,0,0,0,0,0.0


[12 15 18 22 25 27]


,index,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,0,1,BV01,1-BV01,402,46,8,9,19,1
1,1,1,BV02,1-BV02,310,46,3,10,17,0
2,2,1,BV03,1-BV03,244,30,0,2,12,2
3,3,1,BV04,1-BV04,84,7,1,4,23,0
4,4,1,GR01,1-GR01,56,9,6,1,1,1
...,...,...,...,...,...,...,...,...,...,...
4381,155,29,WNO001,29-WNO001,*,8,*,*,*,*
4382,156,29,WNO002,29-WNO002,*,*,*,*,*,*
4383,157,29,WNO003,29-WNO003,24,17,0,0,0,0
4384,158,29,LAKE,29-LAKE,0,0,0,0,0,0


,index,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI
0,0,1,BV01,1-BV01,402,46,8,9,19,1
1,1,1,BV02,1-BV02,310,46,3,10,17,0
2,2,1,BV03,1-BV03,244,30,0,2,12,2
3,3,1,BV04,1-BV04,84,7,1,4,23,0
4,4,1,GR01,1-GR01,56,9,6,1,1,1
...,...,...,...,...,...,...,...,...,...,...
4381,155,29,WNO001,29-WNO001,*,8,*,*,*,*
4382,156,29,WNO002,29-WNO002,*,*,*,*,*,*
4383,157,29,WNO003,29-WNO003,24,17,0,0,0,0
4384,158,29,LAKE,29-LAKE,0,0,0,0,0,0
